In [ ]:
# step 1-> identify oi cross over in trending OI

# first i need oi data of all strikes
# then i will select the trending oi 
# then I will do the sum and check the cross over happening or not 

# Always there are two things 
# 1. check on historical data 
#     for this i need data and i will loop over each date and check the condition 
#         if condition meet print the datetime
# 2. check in live market 
#     fetch the live data and do the same computation 


In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from kite_trade import *
from enctoken import get_kite
kite = get_kite()
import warnings
warnings.filterwarnings(action = 'ignore')
from IPython.display import clear_output
import datetime

# oi data

In [ ]:
def fetch_market_data(expiry_march, start_dt, end_dt, time_frame="minute"):
    df_expiry_hist_data = pd.DataFrame()
    strike_symbol_dict = dict(
        zip(expiry_march.instrument_token, expiry_march["strike_type"])
    )
    inst_expiry = expiry_march["instrument_token"].unique().tolist()
    for i in inst_expiry:
        df = pd.DataFrame(
            kite.historical_data(
                i,
                from_date=start_dt,
                to_date=end_dt,
                interval=time_frame,
                continuous=False,
                oi=True,
            )
        )
        df["strike_type"] = strike_symbol_dict[i]
        df_expiry_hist_data = df_expiry_hist_data.append(df)
    df_expiry_hist_data["date"] = pd.to_datetime(
        df_expiry_hist_data["date"]
    ).dt.tz_localize(None)
    df_expiry_hist_data["date_only"] = pd.to_datetime(
        df_expiry_hist_data["date"].dt.date
    )
    return df_expiry_hist_data


def reorder_column(df):
    strike_prices = sorted(
        set(int(col.split("_")[1]) for col in df.columns if "oi" or "chg" in col)
    )
    reordered_columns = []
    for price in strike_prices:
        reordered_columns.extend(
            [f"oi_{price}_CE", f"chg_{price}_CE",f"chg_{price}_PE", f"oi_{price}_PE" ]
        )
    return reordered_columns


def prev_day_oi(oi_strike, itm_strike, atm_strike, otm_strike):
    for i in range(1, 6):
        day = datetime.datetime.now() - datetime.timedelta(days=i)
        start_dt_prev = day.strftime("%Y-%m-%d")
        end_dt_prev = day.strftime("%Y-%m-%d")
        time_frame = "minute"
        print(start_dt_prev, end_dt_prev)
        df_prev = fetch_market_data(oi_strike, start_dt_prev, end_dt_prev, time_frame)
        print(df_prev.shape)
        if df_prev.empty:
            continue
        else:
            df_prev["strike"] = df_prev["strike_type"].str[:-2].astype(int)
            df_prev["ce_pe"] = df_prev["strike_type"].str[-2:]
            break
    df_prev = (
        df_prev[df_prev["strike"].isin(itm_strike + [atm_strike] + otm_strike)]
        .groupby("strike_type")
        .tail(1)
    )
    return df_prev


In [ ]:
expiry_sp = 'BANKNIFTY2465'
start_dt = datetime.datetime.now().strftime("%Y-%m-%d")
end_dt = datetime.datetime.now().strftime("%Y-%m-%d")
time_frame = "minute"

inst = pd.DataFrame(kite.instruments("NFO"))
expiry_march  = inst[(inst.name == 'BANKNIFTY') & (inst.tradingsymbol.str.contains(expiry_sp))]
expiry_march['strike_type'] = expiry_march['strike'].astype(int).astype(str) + expiry_march['instrument_type']

In [125]:

df_prev = pd.DataFrame()
ltp_dict = kite.ltp([256265, 260105])
nifty_price = ltp_dict["256265"]["last_price"]
banknifty_price = ltp_dict["260105"]["last_price"]
atm_strike = int(banknifty_price // 100 * 100)
otm_strike = [atm_strike + i * 100 for i in range(1, 3)]
itm_strike = [atm_strike - i * 100 for i in range(1, 3)]
print(atm_strike, otm_strike, itm_strike)

oi_strike = expiry_march.query(
    "strike in @otm_strike or strike in @itm_strike or strike == @atm_strike"
)

if df_prev.empty:
    df_prev = prev_day_oi(oi_strike, itm_strike, atm_strike, otm_strike)

df_oi = fetch_market_data(oi_strike, start_dt, end_dt, time_frame)
df_oi["strike"] = df_oi["strike_type"].str[:-2].astype(int)
df_oi["ce_pe"] = df_oi["strike_type"].str[-2:]

df_oi_merge = pd.merge(
    df_oi, df_prev[["strike_type", "oi"]], on="strike_type", suffixes=("_live", "_prev")
).assign(chg=lambda x: x["oi_live"] - x["oi_prev"])
df_oi_merge = df_oi_merge.drop(["oi_prev"], axis=1).rename(columns={"oi_live": "oi"})
df_oi_merge_pivot = (
    df_oi_merge[df_oi_merge["strike"].isin(itm_strike + [atm_strike] + otm_strike)]
    .pivot_table(
        index="date", columns=["strike", "ce_pe"], values=["oi", "chg"], aggfunc="sum"
    )
    .sort_values("date", ascending=False)
)
df_oi_merge_pivot.columns = [
    "_".join([str(i) for i in col]) for col in df_oi_merge_pivot.columns
]
df_oi_merge_pivot[reorder_column(df_oi_merge_pivot)].head()


48900 [49000, 49100] [48800, 48700]
2024-04-09 2024-04-09
(3750, 9)


,oi_48700_CE,chg_48700_CE,chg_48700_PE,oi_48700_PE,oi_48800_CE,chg_48800_CE,chg_48800_PE,oi_48800_PE,oi_48900_CE,chg_48900_CE,chg_48900_PE,oi_48900_PE,oi_49000_CE,chg_49000_CE,chg_49000_PE,oi_49000_PE,oi_49100_CE,chg_49100_CE,chg_49100_PE,oi_49100_PE
date,,,,,,,,,,,,,,,,,,,,
2024-04-10 15:29:00,1004595,-662700,518850,2248275,1579215,-410970,2828250,3856410,3896610,1989720,6227400,6787830,9183495,6023670,7176885,7824705,4026435,2669790,1288545,1482570
2024-04-10 15:28:00,1004595,-662700,518850,2248275,1579215,-410970,2828250,3856410,3896610,1989720,6227400,6787830,9183495,6023670,7176885,7824705,4026435,2669790,1288545,1482570
2024-04-10 15:27:00,1047720,-619575,755325,2484750,1778220,-211965,3312675,4340835,4129140,2222250,7118130,7678560,9856290,6696465,8521425,9169245,4476360,3119715,1441470,1635495
2024-04-10 15:26:00,1047720,-619575,755325,2484750,1778220,-211965,3312675,4340835,4129140,2222250,7118130,7678560,9856290,6696465,8521425,9169245,4476360,3119715,1441470,1635495
2024-04-10 15:25:00,1047720,-619575,755325,2484750,1778220,-211965,3312675,4340835,4129140,2222250,7118130,7678560,9856290,6696465,8521425,9169245,4476360,3119715,1441470,1635495


In [127]:
df_oi.query("strike_type == '48800CE'").tail(5)


,date,open,high,low,close,volume,oi,strike_type,date_only,strike,ce_pe
370,2024-04-10 15:25:00,185.9,186.65,184.8,185.80,175560,1778220,48800CE,2024-04-10,48800,CE
371,2024-04-10 15:26:00,185.8,187.65,185.3,185.65,151590,1778220,48800CE,2024-04-10,48800,CE
372,2024-04-10 15:27:00,185.9,186.05,184.2,185.35,91170,1778220,48800CE,2024-04-10,48800,CE
373,2024-04-10 15:28:00,185.4,186.45,185.1,186.10,80580,1579215,48800CE,2024-04-10,48800,CE
374,2024-04-10 15:29:00,186.1,186.70,185.6,186.40,85245,1579215,48800CE,2024-04-10,48800,CE


# live market oi cross over

In [2]:
expiry_sp = 'BANKNIFTY24410'
inst = pd.DataFrame(kite.instruments("NFO"))
expiry_march  = inst[(inst.name == 'BANKNIFTY') & (inst.tradingsymbol.str.contains(expiry_sp))]
# expiry_march = expiry_march.query("(strike >= 38500 )&( strike <= 40500)")
expiry_march['strike_type'] = expiry_march['strike'].astype(int).astype(str) + expiry_march['instrument_type']


In [3]:
expiry_march.head()

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange,strike_type
2255,11799298,46091,BANKNIFTY2441039000CE,BANKNIFTY,0.0,2024-04-10,39000.0,0.05,15,CE,NFO-OPT,NFO,39000CE
2256,11799554,46092,BANKNIFTY2441039000PE,BANKNIFTY,0.0,2024-04-10,39000.0,0.05,15,PE,NFO-OPT,NFO,39000PE
2257,11800578,46096,BANKNIFTY2441039500CE,BANKNIFTY,0.0,2024-04-10,39500.0,0.05,15,CE,NFO-OPT,NFO,39500CE
2258,11800834,46097,BANKNIFTY2441039500PE,BANKNIFTY,0.0,2024-04-10,39500.0,0.05,15,PE,NFO-OPT,NFO,39500PE
2259,11801090,46098,BANKNIFTY2441040000CE,BANKNIFTY,0.0,2024-04-10,40000.0,0.05,15,CE,NFO-OPT,NFO,40000CE


In [ ]:
# for today's oi cross over check 
# 1. i need the todays data and continously fetch the 1 day before data and take the last day ending data
# 2. then i will check the trending oi cross over 
# 3. then i will check strike cross over 
#     for it i need current spot price
#     and i will check in 1 above and below strike price for cross over 
#     i will plot the 3 strike and keep the oi table if needed


In [37]:
import datetime
import time

def fetch_market_data(expiry_march,start_dt, end_dt, time_frame = 'minute'):
    df_expiry_hist_data = pd.DataFrame()
    strike_symbol_dict = dict(zip(expiry_march.instrument_token,expiry_march['strike_type'] ))
    inst_expiry = expiry_march['instrument_token'].unique().tolist() 
    for i in inst_expiry:
        df = pd.DataFrame(kite.historical_data(i, from_date = start_dt, to_date = end_dt, interval = time_frame, continuous=False, oi=True))
        df['strike_type'] = strike_symbol_dict[i]
        df_expiry_hist_data = df_expiry_hist_data.append(df)
    df_expiry_hist_data['date']  = pd.to_datetime(df_expiry_hist_data['date']).dt.tz_localize(None)
    df_expiry_hist_data['date_only'] = pd.to_datetime(df_expiry_hist_data['date'].dt.date)
    return df_expiry_hist_data



def get_previous_trading_day_data():
    # Check the last five days
    pass


# previous_day_data = get_previous_trading_day_data()
# print(f"Previous trading day's data at 15:27: {previous_day_data}")

# while True:
#     now = datetime.datetime.now()
#     today_data = fetch_market_data(now.strftime('%Y-%m-%d'), now.strftime('%H:%M'))
#     print(f"Today's data at {now.strftime('%H:%M')}: {today_data}")
    
#     # Wait for 60 seconds before the next fetch
#     time.sleep(60)



In [ ]:
import datetime
trending_strike = [46100, 46200, 46300, 46400, 46500, 46600, 46700, 46800, 46900, 47000, 47100, 47200, 47300, 47400, 47500]

# previous day data fetch 
for i in range(1, 6):
        day = datetime.datetime.now() - datetime.timedelta(days=i)
        start_dt = day.strftime('%Y-%m-%d')
        end_dt = day.strftime('%Y-%m-%d')
        time_frame = '3minute'
        df_prev = fetch_market_data(expiry_march,start_dt, end_dt, time_frame)
        if df_prev.empty:
            continue
        else:
               df_prev
        # break
               
    df_bn_oi_data['strike_value'] = df_bn_oi_data['strike_type'].str[:-2]
    df_bn_oi_data['strike_value'] = df_bn_oi_data['strike_value'].astype(int)
    df_bn_oi_data['strike_ce_pe'] = df_bn_oi_data['strike_type'].str[-2:]
    df_bn_oi_data.query('strike_value in @trending_strike', inplace = True)
    df_bn_oi_data_prev_day = df_bn_oi_data_prev_day.groupby(['date', 'strike_ce_pe'])['oi'].sum().reset_index()
    df_pivot_prev = df_bn_oi_data_prev_day.pivot_table(index='date', columns='strike_ce_pe', values='oi')

In [ ]:
# current day data fetch 
now = datetime.datetime.now()
start_dt = now.strftime('%Y-%m-%d')
end_dt = now.strftime('%Y-%m-%d')
time_frame = '3minute'
df_current_day = fetch_market_data(expiry_march,start_dt, end_dt, time_frame)
df_current_day['strike_value'] = df_current_day['strike_type'].str[:-2]
df_current_day['strike_value'] = df_current_day['strike_value'].astype(int)
df_current_day['strike_ce_pe'] = df_current_day['strike_type'].str[-2:]
df_current_day.query('strike_value in @trending_strike', inplace = True)
df_bn_oi_data_current_day = df_current_day.groupby(['date', 'strike_ce_pe'])['oi'].sum().reset_index()
df_pivot_current = df_bn_oi_data_current_day.pivot_table(index='date', columns='strike_ce_pe', values='oi')

In [ ]:
## cross over 
df_pivot = df_pivot_curr.reset_index()
df_pivot.drop(['CE', 'PE'], axis = 1, inplace = True)

# Initialize an empty list to store crossover points
crossovers = []

# Loop through the DataFrame starting from the second row
for i in range(1, len(df_pivot)):
    # Current and previous values for 'chg_ce' and 'chg_pe'
    current_chg_ce = df_pivot.loc[i, 'chg_ce']
    previous_chg_ce = df_pivot.loc[i - 1, 'chg_ce']
    current_chg_pe = df_pivot.loc[i, 'chg_pe']
    previous_chg_pe = df_pivot.loc[i - 1, 'chg_pe']

    # Check for crossover: chg_ce crosses above chg_pe or chg_ce crosses below chg_pe
    if (current_chg_ce > current_chg_pe and previous_chg_ce < previous_chg_pe) or \
       (current_chg_ce < current_chg_pe and previous_chg_ce > previous_chg_pe):
        # Crossover detected; add it to the list
        crossover_date = df_pivot.loc[i, 'date']
        crossovers.append(crossover_date)

# Print all crossover points
for crossover in crossovers:
    print(f"Crossover occurred on: {crossover}")

# If you want to return the crossovers as well, just use `return crossovers` if this is in a function.

In [44]:
from plyer import notification

# Your condition
condition = True  # This is just an example condition. Replace it with your actual condition.

if condition:
    notification.notify(
        title='Akash first notification',
        message='Welcome to trading world',
        app_name='Akash Trading App',
    )

## oi cross over 

In [2]:
df_bn_oi_data = pd.read_parquet("../data/expiry_27march2024_3min.parquet")
df_bn_oi_data = df_bn_oi_data.query("strike_type != '0FUT'")
df_bn_oi_data['date']  = pd.to_datetime(df_bn_oi_data['date']).dt.tz_localize(None)
df_bn_oi_data['date_only'] = pd.to_datetime(df_bn_oi_data['date'].dt.date)
# trending_strike = list(range(46100, 47501, 100))
trending_strike = [46100, 46200, 46300, 46400, 46500, 46600, 46700, 46800, 46900, 47000, 47100, 47200, 47300, 47400, 47500]
# trending_strike = [46700]
df_bn_oi_data['strike_value'] = df_bn_oi_data['strike_type'].str[:-2]
df_bn_oi_data['strike_value'] = df_bn_oi_data['strike_value'].astype(int)
df_bn_oi_data['strike_ce_pe'] = df_bn_oi_data['strike_type'].str[-2:]
df_bn_oi_data.query('strike_value in @trending_strike', inplace = True)

In [3]:
df_bn_oi_data_prev_day = df_bn_oi_data.query("date_only == '2024-03-21'")
df_bn_oi_data_prev_day = df_bn_oi_data_prev_day.groupby(['date', 'strike_ce_pe'])['oi'].sum().reset_index()
df_pivot_prev = df_bn_oi_data_prev_day.pivot_table(index='date', columns='strike_ce_pe', values='oi')

In [4]:
df_pivot_prev.tail()

strike_ce_pe,CE,PE
date,,
2024-03-21 15:15:00,13427790.0,10294725.0
2024-03-21 15:18:00,13233225.0,10125030.0
2024-03-21 15:21:00,12830535.0,9773445.0
2024-03-21 15:24:00,12498615.0,9357075.0
2024-03-21 15:27:00,12304725.0,8987835.0


In [5]:
prev_day_ce_oi = 12304725.0 - (5520975 - 4935675)
prev_day_pe_oi = 8987835.0 - (4918185 - 4573980)

In [6]:
df_bn_oi_data_curr_day = df_bn_oi_data.query("date_only == '2024-03-22'")
df_bn_oi_data_curr_day= df_bn_oi_data_curr_day.groupby(['date', 'strike_ce_pe'])['oi'].sum().reset_index()
df_pivot_curr = df_bn_oi_data_curr_day.pivot_table(index='date', columns='strike_ce_pe', values='oi')

In [7]:
df_pivot_curr['chg_ce'] = df_pivot_curr['CE'] - prev_day_ce_oi
df_pivot_curr['chg_pe'] = df_pivot_curr['PE'] -prev_day_pe_oi

In [8]:
df_pivot_curr.head()

strike_ce_pe,CE,PE,chg_ce,chg_pe
date,,,,
2024-03-22 09:15:00,12028170.0,8963310.0,308745.0,319680.0
2024-03-22 09:18:00,13238385.0,10088415.0,1518960.0,1444785.0
2024-03-22 09:21:00,13866630.0,10891080.0,2147205.0,2247450.0
2024-03-22 09:24:00,14345490.0,11390700.0,2626065.0,2747070.0
2024-03-22 09:27:00,14617845.0,11503155.0,2898420.0,2859525.0


In [9]:
import pandas as pd

df_pivot = df_pivot_curr.reset_index()
df_pivot.drop(['CE', 'PE'], axis = 1, inplace = True)

# Initialize an empty list to store crossover points
crossovers = []

# Loop through the DataFrame starting from the second row
for i in range(1, len(df_pivot)):
    # Current and previous values for 'chg_ce' and 'chg_pe'
    current_chg_ce = df_pivot.loc[i, 'chg_ce']
    previous_chg_ce = df_pivot.loc[i - 1, 'chg_ce']
    current_chg_pe = df_pivot.loc[i, 'chg_pe']
    previous_chg_pe = df_pivot.loc[i - 1, 'chg_pe']

    # Check for crossover: chg_ce crosses above chg_pe or chg_ce crosses below chg_pe
    if (current_chg_ce > current_chg_pe and previous_chg_ce < previous_chg_pe) or \
       (current_chg_ce < current_chg_pe and previous_chg_ce > previous_chg_pe):
        # Crossover detected; add it to the list
        crossover_date = df_pivot.loc[i, 'date']
        crossovers.append(crossover_date)

# Print all crossover points
for crossover in crossovers:
    print(f"Crossover occurred on: {crossover}")

# If you want to return the crossovers as well, just use `return crossovers` if this is in a function.


Crossover occurred on: 2024-03-22 09:18:00
Crossover occurred on: 2024-03-22 09:21:00
Crossover occurred on: 2024-03-22 09:27:00
Crossover occurred on: 2024-03-22 09:36:00


In [10]:
import plotly.express as px
import pandas as pd

# Assuming df_bn_oi_data is your DataFrame with the necessary data
# df_pivot = df_bn_oi_data.groupby(['date', 'strike_ce_pe'])['oi'].sum().reset_index().pivot(index='date', columns='strike_ce_pe', values='oi')

# Reset index to make 'date' a column again, which is necessary for Plotly


# Plotting with Plotly
fig = px.line(df_pivot, x='date', y=df_pivot.columns,
              title='OI Pulse',
              labels={'value': 'OI', 'variable': 'Option Type'})

# Adding more interactive features like hover information
fig.update_traces(mode='lines+markers')
fig.update_layout(hovermode='x unified')

fig.show()


In [73]:
df_pivot

strike_ce_pe,date,CE,PE
0,2024-03-22 09:15:00,12028170.0,8963310.0
1,2024-03-22 09:18:00,13238385.0,10088415.0
2,2024-03-22 09:21:00,13866630.0,10891080.0
3,2024-03-22 09:24:00,14345490.0,11390700.0
4,2024-03-22 09:27:00,14617845.0,11503155.0
...,...,...,...
120,2024-03-22 15:15:00,12484230.0,14412510.0
121,2024-03-22 15:18:00,12233190.0,14141160.0
122,2024-03-22 15:21:00,11942310.0,13658610.0
123,2024-03-22 15:24:00,11577585.0,13068345.0


In [58]:
import pandas as pd
import plotly.graph_objs as go

# Assuming df_pivot is your DataFrame with 'date', 'CE', and 'PE' columns

# Convert 'date' column to datetime
df_pivot['date'] = pd.to_datetime(df_pivot['date'])

# Sort the DataFrame by 'date' just in case it's not sorted
df_pivot = df_pivot.sort_values('date')

# Calculate the change in 'CE' and 'PE'
df_pivot['change_in_CE'] = df_pivot['CE'].diff()
df_pivot['change_in_PE'] = df_pivot['PE'].diff()

# Create a Plotly graph
fig = go.Figure()

# Add traces for 'change_in_CE' and 'change_in_PE'
fig.add_trace(go.Scatter(x=df_pivot['date'], y=df_pivot['change_in_CE'],
                         mode='lines+markers', name='Change in Call OI'))
fig.add_trace(go.Scatter(x=df_pivot['date'], y=df_pivot['change_in_PE'],
                         mode='lines+markers', name='Change in Put OI'))

# Update layout for a better look
fig.update_layout(title='Change in OI for CE and PE',
                  xaxis_title='Date',
                  yaxis_title='Change in OI',
                  hovermode='x unified')

# Show plot
fig.show()


In [56]:
df_pivot

strike_ce_pe,date,CE,PE
0,2024-03-21 09:15:00,7026255.0,4429050.0
1,2024-03-21 09:18:00,7703340.0,5392590.0
2,2024-03-21 09:21:00,8208375.0,6696285.0
3,2024-03-21 09:24:00,8786385.0,7280700.0
4,2024-03-21 09:27:00,9250770.0,7973070.0
...,...,...,...
120,2024-03-21 15:15:00,13427790.0,10294725.0
121,2024-03-21 15:18:00,13233225.0,10125030.0
122,2024-03-21 15:21:00,12830535.0,9773445.0
123,2024-03-21 15:24:00,12498615.0,9357075.0


In [49]:
df_bn_oi_data.query("strike_value == 46600 & strike_ce_pe == 'CE'").tail(10)

,date,open,high,low,close,volume,oi,strike_type,date_only,strike_value,strike_ce_pe
115,2024-03-21 15:00:00,477.85,481.85,459.75,467.70,133020.0,592890.0,46600CE,2024-03-21,46600,CE
116,2024-03-21 15:03:00,466.25,472.90,461.15,468.75,90975.0,602280.0,46600CE,2024-03-21,46600,CE
117,2024-03-21 15:06:00,468.25,469.95,445.95,459.90,158145.0,601545.0,46600CE,2024-03-21,46600,CE
118,2024-03-21 15:09:00,459.90,471.95,459.40,469.75,128505.0,593700.0,46600CE,2024-03-21,46600,CE
119,2024-03-21 15:12:00,469.85,471.00,462.00,466.90,78765.0,582495.0,46600CE,2024-03-21,46600,CE
120,2024-03-21 15:15:00,466.50,470.65,460.90,467.00,101475.0,558465.0,46600CE,2024-03-21,46600,CE
121,2024-03-21 15:18:00,467.40,469.45,461.40,462.60,80955.0,535725.0,46600CE,2024-03-21,46600,CE
122,2024-03-21 15:21:00,462.95,467.00,460.60,464.15,81375.0,504615.0,46600CE,2024-03-21,46600,CE
123,2024-03-21 15:24:00,464.15,465.50,458.40,463.60,92700.0,471855.0,46600CE,2024-03-21,46600,CE
124,2024-03-21 15:27:00,463.60,471.05,460.00,470.95,86055.0,455295.0,46600CE,2024-03-21,46600,CE


In [2]:
expiry_sp = 'BANKNIFTY24MAR'
inst = pd.DataFrame(kite.instruments("NFO"))
expiry_march  = inst[(inst.name == 'BANKNIFTY') & (inst.tradingsymbol.str.contains(expiry_sp))]
# expiry_march = expiry_march.query("(strike >= 38500 )&( strike <= 40500)")
expiry_march['strike_type'] = expiry_march['strike'].astype(int).astype(str) + expiry_march['instrument_type']

In [24]:
inst.query("name =='NIFTY' ")

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,13368834,52222,NIFTY24APRFUT,NIFTY,0.0,2024-04-25,0.0,0.05,50,FUT,NFO-FUT,NFO
1,9372674,36612,NIFTY24MARFUT,NIFTY,0.0,2024-03-28,0.0,0.05,50,FUT,NFO-FUT,NFO
2,12014082,46930,NIFTY24MAYFUT,NIFTY,0.0,2024-05-30,0.0,0.05,50,FUT,NFO-FUT,NFO
558,15440642,60315,NIFTY24MAR14000CE,NIFTY,0.0,2024-03-28,14000.0,0.05,50,CE,NFO-OPT,NFO
559,15440898,60316,NIFTY24MAR14000PE,NIFTY,0.0,2024-03-28,14000.0,0.05,50,PE,NFO-OPT,NFO
...,...,...,...,...,...,...,...,...,...,...,...,...
2076,17352962,67785,NIFTY28DEC25000PE,NIFTY,0.0,2028-12-28,25000.0,0.05,50,PE,NFO-OPT,NFO
2077,17353218,67786,NIFTY28DEC26000CE,NIFTY,0.0,2028-12-28,26000.0,0.05,50,CE,NFO-OPT,NFO
2078,17353474,67787,NIFTY28DEC26000PE,NIFTY,0.0,2028-12-28,26000.0,0.05,50,PE,NFO-OPT,NFO
2079,17353730,67788,NIFTY28DEC27000CE,NIFTY,0.0,2028-12-28,27000.0,0.05,50,CE,NFO-OPT,NFO


In [9]:
i = 9372674 #9372418 #
start_dt = '2024-03-01'
end_dt = '2024-03-27'
time_frame = '3minute'
df = pd.DataFrame(kite.historical_data(i, from_date = start_dt, to_date = end_dt, interval = time_frame, continuous=False, oi=True))

In [11]:
df.to_parquet('data/nifty_march_futures.parquet')

In [85]:
df['date']  = pd.to_datetime(df['date']).dt.tz_localize(None)
df['date_only'] = pd.to_datetime(df['date'].dt.date)

## gap theory

In [ ]:
import pandas as pd

# Assuming 'df' is your DataFrame with columns: date, open, high, low, close

# Convert the 'date' column to datetime format and filter the entries between 9:30 and 15:00
# df['date'] = pd.to_datetime(df['date'])
# filter for date 20/03/2024



# Initialize an empty list to store the gap information
gaps = []

# Iterate over the filtered DataFrame to find gaps
for j in df.date_only.unique():
    print(j)
    df_temp = df[df.date_only == j]
    filtered_df = df_temp[df_temp['date'].dt.time.between(pd.to_datetime('09:30').time(), pd.to_datetime('15:00').time())].reset_index(drop = True)
    for i in range(1,len(filtered_df)):
        previous_row = filtered_df.iloc[i - 1]
        current_row = filtered_df.iloc[i]

        # Check if the previous candle is green
        if previous_row['close'] > previous_row['open']:
            # Check for a gap up
            if current_row['low'] > previous_row['high'] + 2:
                gaps.append({
                    'type': 'gap up',
                    'from': previous_row['high'],
                    'to': current_row['open'],
                    'index': i  # Index of the current row in the filtered DataFrame
                })

        # Check if the previous candle is red
        elif previous_row['close'] < previous_row['open']:
            # Check for a gap down
            if current_row['high'] < previous_row['low'] - 2:
                gaps.append({
                    'type': 'gap down',
                    'from': previous_row['low'],
                    'to': current_row['open'],
                    'index': i  # Index of the current row in the filtered DataFrame
                })
        # break
    
# The 'gaps' list now contains the details of all identified gaps


## golden cross over

In [79]:
df.head()

,date,open,high,low,close,volume,oi
0,2024-03-01 09:15:00+05:30,22203.00,22250.00,22180.20,22243.95,357550,13084800
1,2024-03-01 09:18:00+05:30,22243.95,22244.95,22232.45,22239.75,140950,13069950
2,2024-03-01 09:21:00+05:30,22239.75,22248.60,22236.20,22242.30,120400,13070650
3,2024-03-01 09:24:00+05:30,22242.30,22254.90,22240.80,22248.20,129050,13080800
4,2024-03-01 09:27:00+05:30,22248.20,22253.45,22238.20,22245.00,100950,13027100


In [86]:
import pandas as pd
import numpy as np
import ta

def calculate_supertrend(df, period=10, multiplier=2):
    hl2 = (df['high'] + df['low']) / 2
    atr = ta.volatility.average_true_range(df['high'], df['low'], df['close'], window=period)
    upperband = hl2 + (multiplier * atr)
    lowerband = hl2 - (multiplier * atr)
    supertrend = np.zeros(len(df))

    for i in range(1, len(df)):
        if df['close'][i-1] <= supertrend[i-1]:
            supertrend[i] = min(upperband[i], supertrend[i-1])
        else:
            supertrend[i] = max(lowerband[i], supertrend[i-1])
        if df['close'][i] > supertrend[i]:
            supertrend[i] = upperband[i]
        elif df['close'][i] < supertrend[i]:
            supertrend[i] = lowerband[i]

    return supertrend

def calculate_daily_vwap(df):
    df['cum_vol_price'] = (df['volume'] * df['close']).cumsum()
    df['cum_volume'] = df['volume'].cumsum()
    df['VWAP'] = df['cum_vol_price'] / df['cum_volume']
    return df

def calculate_indicators(df):
    df['VWMA20'] = ta.trend.sma_indicator(df['close'] * df['volume'], window=20) / ta.trend.sma_indicator(df['volume'], window=20)
    df = df.groupby('date_only').apply(calculate_daily_vwap)
    df['SuperTrend'] = calculate_supertrend(df, period=10, multiplier=2)
    return df

def generate_signals(df):
    df = calculate_indicators(df)
    df['Signal'] = ((df['VWMA20'] > df['VWAP']) & (df['SuperTrend'] > df['VWAP'])) | ((df['VWMA20'] < df['VWAP']) & (df['SuperTrend'] < df['VWAP']))
    # signals = df[df['Signal']].copy()
    return df


signals = generate_signals(df)
# print(signals)


In [88]:
signals.tail(10)

,date,open,high,low,close,volume,oi,date_only,VWMA20,cum_vol_price,cum_volume,VWAP,SuperTrend,Signal
1650,2024-03-20 15:00:00,21915.00,21916.40,21896.20,21897.55,103100,11906750,2024-03-20,21907.435977,1.305767e+11,5960400,21907.376540,21868.013702,False
1651,2024-03-20 15:03:00,21897.00,21907.95,21889.70,21898.90,46350,11917600,2024-03-20,21906.646546,1.315917e+11,6006750,21907.311132,21936.932668,False
1652,2024-03-20 15:06:00,21901.00,21908.70,21900.00,21908.40,47350,11944150,2024-03-20,21905.804739,1.326291e+11,6054100,21907.319649,21868.093098,True
1653,2024-03-20 15:09:00,21906.35,21914.00,21906.35,21910.00,42700,11954300,2024-03-20,21905.954934,1.335647e+11,6096800,21907.338421,21944.336211,False
1654,2024-03-20 15:12:00,21910.00,21919.60,21908.75,21914.90,42500,11968850,2024-03-20,21905.966957,1.344960e+11,6139300,21907.390767,21881.259910,True
1655,2024-03-20 15:15:00,21915.65,21918.30,21910.50,21917.00,52450,11990050,2024-03-20,21905.655598,1.356456e+11,6191750,21907.472166,21945.583581,False
1656,2024-03-20 15:18:00,21916.55,21923.90,21910.85,21916.50,33850,12003500,2024-03-20,21905.334387,1.363875e+11,6225600,21907.521253,21886.699777,True
1657,2024-03-20 15:21:00,21916.50,21922.60,21915.85,21916.65,38650,12015100,2024-03-20,21905.951501,1.372345e+11,6264250,21907.577576,21948.182701,False
1658,2024-03-20 15:24:00,21917.00,21922.20,21915.05,21916.00,37600,12017350,2024-03-20,21906.856145,1.380586e+11,6301850,21907.627829,21891.133069,True
1659,2024-03-20 15:27:00,21915.00,21920.00,21914.90,21919.00,50900,12015350,2024-03-20,21907.626953,1.391743e+11,6352750,21907.718946,21943.212738,False


## PRICE AT SUPER TREND AND VWAP

In [99]:
def date_format(df):
    df['date']  = pd.to_datetime(df['date']).dt.tz_localize(None)
    df['date_only'] = pd.to_datetime(df['date'].dt.date)
    return df 

In [110]:
import talib
expiry_sp = 'BANKNIFTY24430'
inst = pd.DataFrame(kite.instruments("NFO"))
expiry_march  = inst[(inst.name == 'BANKNIFTY') & (inst.tradingsymbol.str.contains(expiry_sp))]
# expiry_march = expiry_march.query("(strike >= 38500 )&( strike <= 40500)")
expiry_march['strike_type'] = expiry_march['strike'].astype(int).astype(str) + expiry_march['instrument_type']

In [118]:
expiry_march[expiry_march['tradingsymbol'].str.contains("48200")]

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange,strike_type
2059,12679938,49531,BANKNIFTY2443048200CE,BANKNIFTY,0.0,2024-04-30,48200.0,0.05,15,CE,NFO-OPT,NFO,48200CE
2060,12680194,49532,BANKNIFTY2443048200PE,BANKNIFTY,0.0,2024-04-30,48200.0,0.05,15,PE,NFO-OPT,NFO,48200PE


In [119]:
bn_token = 12012290
start_dt = '2024-04-25'
end_dt = '2024-04-26'
time_frame = '3minute'
df = pd.DataFrame(kite.historical_data(bn_token, from_date = start_dt, to_date = end_dt, interval = time_frame, continuous=False, oi=True))

call_token = 12679938
put_token = 12680194
df_call_atm = pd.DataFrame(kite.historical_data(call_token, from_date = start_dt, to_date = end_dt, interval = time_frame, continuous=False, oi=True))
df_put_atm = pd.DataFrame(kite.historical_data(put_token, from_date = start_dt, to_date = end_dt, interval = time_frame, continuous=False, oi=True))

In [120]:
df = date_format(df)
df_call_atm = date_format(df_call_atm)
df_put_atm = date_format(df_put_atm)

In [121]:
df.head()

,date,open,high,low,close,volume,oi,date_only
0,2024-04-25 09:15:00,48005.40,48236.15,47999.90,48157.95,136770,1639500,2024-04-25
1,2024-04-25 09:18:00,48161.00,48163.20,48130.00,48155.30,49470,1649085,2024-04-25
2,2024-04-25 09:21:00,48153.15,48164.80,48139.80,48142.00,22890,1657440,2024-04-25
3,2024-04-25 09:24:00,48142.00,48169.95,48142.00,48163.80,21135,1660470,2024-04-25
4,2024-04-25 09:27:00,48162.25,48195.75,48162.25,48189.20,18930,1659585,2024-04-25


In [122]:
df_merged = df.merge(df_call_atm, on = 'date', suffixes = ("_fut", '_strike'))

In [123]:
import pandas as pd

# Assuming df_merged is your DataFrame with columns 'date', 'close_fut', and 'close_strike'

# Calculate the day-over-day changes for 'close_fut' and 'close_strike'
df_merged['change_close_fut'] = df_merged['close_fut'].diff()
df_merged['change_close_strike'] = df_merged['close_strike'].diff()

# Calculate the sensitivity of 'close_strike' relative to 'close_fut'
# We'll avoid division by zero by replacing 0 with NaN in the denominator
df_merged['sensitivity'] = df_merged['change_close_strike'] / df_merged['change_close_fut'].replace({0: pd.NA})


# Optional: if you want to know the sensitivity for a specific 1 point change in 'close_fut',
# you can calculate the average sensitivity where 'change_close_fut' is non-zero
average_sensitivity = df_merged[df_merged['change_close_fut'] != 0]['sensitivity'].mean()


In [126]:
df_merged[
    ['date','close_fut','close_strike','change_close_fut','change_close_strike','sensitivity']].tail(10)

,date,close_fut,close_strike,change_close_fut,change_close_strike,sensitivity
221,2024-04-26 14:03:00,48452.20,357.50,1.30,1.10,0.846154
222,2024-04-26 14:06:00,48451.90,355.85,-0.30,-1.65,5.5
223,2024-04-26 14:09:00,48495.05,373.95,43.15,18.10,0.419467
224,2024-04-26 14:12:00,48490.25,372.30,-4.80,-1.65,0.34375
225,2024-04-26 14:15:00,48542.80,403.60,52.55,31.30,0.595623
226,2024-04-26 14:18:00,48527.80,393.25,-15.00,-10.35,0.69
227,2024-04-26 14:21:00,48505.65,379.35,-22.15,-13.90,0.62754
228,2024-04-26 14:24:00,48485.10,371.60,-20.55,-7.75,0.377129
229,2024-04-26 14:27:00,48471.10,364.00,-14.00,-7.60,0.542857
230,2024-04-26 14:30:00,48492.00,375.45,20.90,11.45,0.547847


In [90]:
df_merged

,date,open_fut,high_fut,low_fut,close_fut,volume_fut,oi_fut,date_only_fut,open_sp,high_sp,low_sp,close_sp,volume_sp,oi_sp,date_only_sp
0,2024-04-25 09:15:00,48005.40,48236.15,47999.90,48157.95,136770,1639500,2024-04-25,161.90,211.60,142.05,171.95,294375,275475,2024-04-25
1,2024-04-25 09:18:00,48161.00,48163.20,48130.00,48155.30,49470,1649085,2024-04-25,171.95,174.75,157.20,160.95,209085,336420,2024-04-25
2,2024-04-25 09:21:00,48153.15,48164.80,48139.80,48142.00,22890,1657440,2024-04-25,160.85,166.75,154.45,157.55,132000,374400,2024-04-25
3,2024-04-25 09:24:00,48142.00,48169.95,48142.00,48163.80,21135,1660470,2024-04-25,157.55,165.95,155.25,165.05,141435,401685,2024-04-25
4,2024-04-25 09:27:00,48162.25,48195.75,48162.25,48189.20,18930,1659585,2024-04-25,165.05,183.10,164.70,178.95,161685,423285,2024-04-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,2024-04-26 13:03:00,48638.85,48640.00,48622.00,48629.80,2880,1821315,2024-04-26,336.70,339.95,330.75,338.60,177960,1671945,2024-04-26
202,2024-04-26 13:06:00,48627.40,48639.75,48611.00,48613.35,3885,1821330,2024-04-26,338.65,340.65,325.85,332.25,228060,1681230,2024-04-26
203,2024-04-26 13:09:00,48615.75,48639.00,48613.35,48630.15,1875,1821840,2024-04-26,332.25,339.95,330.00,338.25,186660,1663995,2024-04-26
204,2024-04-26 13:12:00,48630.15,48660.00,48630.15,48645.90,9000,1822500,2024-04-26,338.25,353.15,337.40,343.00,417405,1670610,2024-04-26


In [ ]:
df = df[df.date_only == "2024-04-26"]
df.set_index('date', inplace=True)

# VWAP Calculation (Manual)
cumulative_vp = (df['volume'] * (df['high'] + df['low'] + df['close']) / 3).cumsum()
cumulative_volume = df['volume'].cumsum()
df['vwap'] = cumulative_vp / cumulative_volume

# Calculate ATR using TA-Lib
df['atr'] = talib.ATR(df['high'].values, df['low'].values, df['close'].values, timeperiod=10)

# Supertrend Calculation
factor = 2
df['basic_upperband'] = (df['high'] + df['low']) / 2 + (factor * df['atr'])
df['basic_lowerband'] = (df['high'] + df['low']) / 2 - (factor * df['atr'])
df['supertrend'] = 0.0
df['in_uptrend'] = True

for current in range(1, len(df.index)):
    previous = current - 1
    if df['close'][current] > df['basic_upperband'][previous]:
        df['in_uptrend'][current] = True
    elif df['close'][current] < df['basic_lowerband'][previous]:
        df['in_uptrend'][current] = False

    if df['in_uptrend'][current]:
        df['supertrend'][current] = df['basic_lowerband'][current]
    else:
        df['supertrend'][current] = df['basic_upperband'][current]

In [66]:
df.round(2).tail(5)

,open,high,low,close,volume,oi,date_only,vwap,atr,basic_upperband,basic_lowerband,supertrend,in_uptrend
date,,,,,,,,,,,,,
2024-04-26 12:48:00,48596.00,48599.00,48583.35,48593.95,5835,1817385,2024-04-26,48619.72,29.79,48650.76,48531.59,48531.59,True
2024-04-26 12:51:00,48593.95,48646.00,48593.10,48630.90,10860,1816890,2024-04-26,48619.76,32.10,48683.75,48555.35,48555.35,True
2024-04-26 12:54:00,48638.00,48650.00,48634.85,48644.00,14130,1818525,2024-04-26,48620.09,30.80,48704.03,48580.82,48580.82,True
2024-04-26 12:57:00,48638.65,48650.00,48615.00,48628.65,8760,1817610,2024-04-26,48620.19,31.22,48694.94,48570.06,48570.06,True
2024-04-26 13:00:00,48628.65,48628.65,48620.05,48620.95,525,1817610,2024-04-26,48620.19,28.96,48682.27,48566.43,48566.43,True


In [ ]:
import pandas as pd
import numpy as np
import talib

# # Example DataFrame
# data = {
#     'date': pd.date_range(start='2023-01-01', periods=100, freq='D'),
#     'open': np.random.random(100) * 100,
#     'high': np.random.random(100) * 100,
#     'low': np.random.random(100) * 100,
#     'close': np.random.random(100) * 100,
#     'volume': np.random.randint(100, 1000, 100)
# }

# df = pd.DataFrame(data)







In [5]:
inst

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,12014082,46930,NIFTY24MAYFUT,NIFTY,0.0,2024-05-30,0.0,0.05,25,FUT,NFO-FUT,NFO
1,8961026,35004,NIFTY24JUNFUT,NIFTY,0.0,2024-06-27,0.0,0.05,25,FUT,NFO-FUT,NFO
2,8961794,35007,NIFTY24JULFUT,NIFTY,0.0,2024-07-25,0.0,0.05,25,FUT,NFO-FUT,NFO
3,12012290,46923,BANKNIFTY24MAYFUT,BANKNIFTY,0.0,2024-05-29,0.0,0.05,15,FUT,NFO-FUT,NFO
4,8965122,35020,BANKNIFTY24JUNFUT,BANKNIFTY,0.0,2024-06-26,0.0,0.05,15,FUT,NFO-FUT,NFO
...,...,...,...,...,...,...,...,...,...,...,...,...
34230,37560578,146721,ZYDUSLIFE24JUL1060PE,ZYDUSLIFE,0.0,2024-07-25,1060.0,0.05,900,PE,NFO-OPT,NFO
34231,37560834,146722,ZYDUSLIFE24JUL1080CE,ZYDUSLIFE,0.0,2024-07-25,1080.0,0.05,900,CE,NFO-OPT,NFO
34232,37561090,146723,ZYDUSLIFE24JUL1080PE,ZYDUSLIFE,0.0,2024-07-25,1080.0,0.05,900,PE,NFO-OPT,NFO
34233,37561858,146726,ZYDUSLIFE24JUL1120CE,ZYDUSLIFE,0.0,2024-07-25,1120.0,0.05,900,CE,NFO-OPT,NFO


## stock selection 

first get list of shares to check 
1. eod analyser, futures data at day level to check last 3 to 4 days pattern
2. futures data of current day, to look some buildup in the morning
3. future price data to check OL, OH
4. future price to check how many high or low breaks


In [7]:
from tqdm import tqdm
def fetch_market_data(kite, expiry_march, start_dt, end_dt, time_frame="minute"):
    df_expiry_hist_data = pd.DataFrame()
    strike_symbol_dict = dict(
        zip(expiry_march.instrument_token, expiry_march["name"])
    )
    inst_expiry = expiry_march["instrument_token"].unique().tolist()
    for i in tqdm(inst_expiry):
        try:
            df = pd.DataFrame(
                kite.historical_data(
                    i,
                    from_date=start_dt,
                    to_date=end_dt,
                    interval=time_frame,
                    continuous=False,
                    oi=True,
                )
            )
            df["symbol"] = strike_symbol_dict[i]
            df_expiry_hist_data = df_expiry_hist_data.append(df)
        except Exception as e:
            print(e)
            continue
    df_expiry_hist_data["date"] = pd.to_datetime(
        df_expiry_hist_data["date"]
    ).dt.tz_localize(None)
    df_expiry_hist_data["date_only"] = pd.to_datetime(
        df_expiry_hist_data["date"].dt.date
    )
    return df_expiry_hist_data

In [6]:

def classify_action(row):
    if pd.isna(row['oi_diff']) or pd.isna(row['close_diff']):
        return None  # Ignore the first row for each symbol as it has no previous day to compare
    if row['oi_diff'] > 0 and row['close_diff'] > 0:
        return 'Long Buildup'
    elif row['oi_diff'] > 0 and row['close_diff'] < 0:
        return 'Short Buildup'
    elif row['oi_diff'] < 0 and row['close_diff'] > 0:
        return 'Short Covering'
    elif row['oi_diff'] < 0 and row['close_diff'] < 0:
        return 'Long Unwinding'
    else:
        return 'No Change'  # This covers cases where either oi or close doesn't change


def get_buildup_tag(df_data):
    # Sorting by symbol and date to ensure the data is in the right order
    df_data.sort_values(by=['symbol', 'date'], inplace=True)

    # Calculate differences and percentage changes
    df_data['oi_diff'] = df_data.groupby('symbol')['oi'].diff()
    df_data['oi_pct_chg'] = df_data.groupby('symbol')['oi'].pct_change() * 100
    df_data['close_diff'] = df_data.groupby('symbol')['close'].diff()
    df_data['close_pct_chg'] = df_data.groupby('symbol')['close'].pct_change() * 100
    # Apply function
    df_data['Market_Action'] = df_data.apply(classify_action, axis=1)

    df_data = df_data.round(2)
    return df_data


def get_majority_buildup_flag(df_titan):
    # Sample df_titan loading
    df_titan['volume'] = pd.to_numeric(df_titan['volume'])

    # Calculate percentage change in volume
    # df_titan['perc_change_volume'] = df_titan.groupby('symbol')['volume'].pct_change() * 100
    df_titan['perc_change_volume'] = df_titan['volume']
    mean_volume = df_titan['perc_change_volume'].mean()
    std_volume = df_titan['perc_change_volume'].std()
    df_titan['std_flag'] = ((df_titan['perc_change_volume'] > mean_volume + 2 * std_volume) | (df_titan['perc_change_volume'] < mean_volume - 2 * std_volume))

    Q1 = df_titan['perc_change_volume'].quantile(0.25)
    Q3 = df_titan['perc_change_volume'].quantile(0.75)
    IQR = Q3 - Q1
    df_titan['iqr_flag'] = ((df_titan['perc_change_volume'] < Q1 - 1.5 * IQR) | (df_titan['perc_change_volume'] > Q3 + 1.5 * IQR))


    df_titan['z_score'] = (df_titan['perc_change_volume'] - mean_volume) / std_volume
    df_titan['z_flag'] = (df_titan['z_score'].abs() > 2)

    df_titan['majority_flag'] = df_titan[['std_flag', 'iqr_flag', 'z_flag']].sum(axis=1) >= 2
    return df_titan

In [10]:
inst = pd.DataFrame(kite.instruments("NFO"))
april_fut_inst = inst[inst["tradingsymbol"].str.contains('24JUNFUT')]

start_dt = "2024-0-20"
end_dt = "2024-05-07" 
time_frame = "day"
df_day = fetch_market_data(kite,april_fut_inst, start_dt, end_dt, time_frame=time_frame)

100%|██████████| 187/187 [00:10<00:00, 18.44it/s]


In [54]:
df_day.head()

,date,open,high,low,close,volume,oi,symbol,date_only
0,2024-04-22,22325.00,22531.0,22325.00,22496.55,2541350,5333650,NIFTY,2024-04-22
1,2024-04-23,22549.95,22550.0,22465.00,22485.70,3431850,7013650,NIFTY,2024-04-23
2,2024-04-24,22535.00,22596.0,22479.05,22509.75,4705350,8788750,NIFTY,2024-04-24
3,2024-04-25,22435.20,22715.0,22435.15,22653.95,11645250,11448050,NIFTY,2024-04-25
4,2024-04-26,22684.00,22705.0,22530.00,22556.00,6242950,10399450,NIFTY,2024-04-26


In [55]:
start_dt_td = "2024-04-20"
end_dt_td = "2024-05-07" 
time_frame = "30minute"
df_data_min = fetch_market_data(kite,april_fut_inst, start_dt_td, end_dt_td, time_frame=time_frame)

  0%|          | 0/187 [00:00<?, ?it/s]

100%|██████████| 187/187 [00:13<00:00, 14.26it/s]


In [56]:
date_list = ["2024-05-06", "2024-05-07"]
# [df_data_min['symbol'] == "TITAN"].
df_stock_select = pd.DataFrame()
for sy in tqdm(df_data_min.symbol.unique()):
    try:
        df_day_ind = df_day.query("symbol == @sy")
        df_day_ind = df_day_ind.sort_values('date_only', ascending = False)

        df_titan = df_data_min.query("symbol ==@sy")
        df_titan = df_titan[df_titan['date_only'].isin(date_list)]

        # current day fut buildup 
        df_titan = get_buildup_tag(df_titan)
        df_titan = get_majority_buildup_flag(df_titan)

        # OH and OL logic
        df_titan = df_titan.query("date_only == '2024-05-07'")
        df_titan['OL'] = df_titan[df_titan['date'].dt.time.between(pd.to_datetime('09:15').time(), pd.to_datetime('09:15').time())].apply(lambda x: x['open'] == x['low'],axis =1 )
        df_titan['OH'] = df_titan[df_titan['date'].dt.time.between(pd.to_datetime('09:15').time(), pd.to_datetime('09:15').time())].apply(lambda x: x['open'] == x['high'],axis =1 )
        df_titan['OL'] = df_titan['OL'].ffill()
        df_titan['OH'] = df_titan['OH'].ffill()

        # 9 day high and low breakout logic
        df_min_temp = df_titan.copy()
        df_min_temp = df_min_temp[df_min_temp['date'].dt.time.between(pd.to_datetime('09:15').time(), pd.to_datetime('10:00').time())]
        df_min_temp = df_min_temp.resample('D', on='date').agg({
            'open': 'first',
            'high': 'max',
            'low': 'min',
            'close': 'last'
        }).dropna().reset_index()

        today_high = df_min_temp['high'].iloc[0]
        today_low = df_min_temp['low'].iloc[0]
        df_prev = df_day_ind[df_day_ind["date"] != "2024-05-06"]
        # Initialize counters for consecutive breaks
        consecutive_high_breaks = 0
        consecutive_low_breaks = 0

        # Iterate over the rows in reverse (from most recent to least)
        for index, row in df_prev.iloc[:].iterrows():
            if row['high'] < today_high:
                consecutive_high_breaks += 1
            else:
                break  # Stop counting if a day does not break today's high

        for index, row in df_prev.iloc[:].iterrows():

            if row['low'] > today_low:
                consecutive_low_breaks += 1
            else:
                break  # Stop counting if a day does not break today's low
        df_titan['day_count_high'] = consecutive_high_breaks
        df_titan['day_count_low'] = consecutive_low_breaks
        df_stock_select = df_stock_select.append(df_titan)
        # break
    except:
        print(sy)
        continue


  0%|          | 0/187 [00:00<?, ?it/s]

100%|██████████| 187/187 [00:10<00:00, 18.30it/s]


In [57]:
df_stock_select.to_clipboard(index = False)

In [49]:
df_stock_select.query("OL == True").symbol.unique()

array(['ABB', 'APOLLOHOSP', 'AXISBANK', 'BALRAMCHIN', 'BATAINDIA',
       'COALINDIA', 'CROMPTON', 'DEEPAKNTR', 'DLF', 'EICHERMOT',
       'GODREJPROP', 'HDFCBANK', 'INDHOTEL', 'JUBLFOOD', 'METROPOLIS',
       'MFSL', 'MIDCPNIFTY', 'NMDC', 'POWERGRID', 'RELIANCE', 'SBILIFE',
       'SIEMENS', 'SRF', 'SYNGENE'], dtype=object)

In [50]:
df_stock_select.query("OH == True").symbol.unique()

array(['ABCAPITAL', 'ABFRL', 'ADANIENT', 'ALKEM', 'APOLLOTYRE',
       'ASIANPAINT', 'ASTRAL', 'BALKRISIND', 'BHARATFORG', 'BIOCON',
       'CANFINHOME', 'CHOLAFIN', 'CUB', 'CUMMINSIND', 'DABUR',
       'FEDERALBNK', 'GMRINFRA', 'GRASIM', 'HDFCLIFE', 'ICICIBANK', 'IGL',
       'INDIGO', 'JINDALSTEL', 'LTF', 'LTTS', 'LUPIN', 'MRF', 'PAGEIND',
       'PETRONET', 'PFC', 'POLYCAB', 'PVRINOX', 'RBLBANK', 'SHRIRAMFIN',
       'VEDL'], dtype=object)

In [52]:
df_stock_select.to_excel("stock_select.xlsx",index = False)

In [116]:
df_stock_select.query("OH == True").query("majority_flag == True").query("day_count_high == 1").query("day_count_low == 1").query("Market_Action == 'Long Buildup'")

,date,open,high,low,close,volume,oi,symbol,date_only,oi_diff,...,perc_change_volume,std_flag,iqr_flag,z_score,z_flag,majority_flag,OL,OH,day_count_high,day_count_low
625,2024-04-12 11:20:00,4840.75,4848.00,4840.00,4841.20,20200,1086600,ALKEM,2024-04-12,6600.0,...,20200,True,True,3.291733,True,True,False,True,0,1
628,2024-04-12 11:35:00,4837.00,4847.00,4837.00,4838.30,23400,1118400,ALKEM,2024-04-12,14800.0,...,23400,True,True,3.927407,True,True,False,True,0,1
669,2024-04-12 15:00:00,4767.75,4767.75,4743.45,4750.00,17200,1135000,ALKEM,2024-04-12,200.0,...,17200,True,True,2.695788,True,True,False,True,0,1
600,2024-04-12 09:15:00,6490.50,6490.50,6437.30,6471.20,29875,1705125,APOLLOHOSP,2024-04-12,-6250.0,...,29875,True,True,4.826337,True,True,False,True,0,1
601,2024-04-12 09:20:00,6465.95,6465.95,6446.20,6465.65,15500,1711375,APOLLOHOSP,2024-04-12,6250.0,...,15500,True,True,2.002626,True,True,False,True,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674,2024-04-12 15:25:00,3628.90,3632.00,3626.45,3627.10,82425,5798100,TITAN,2024-04-12,1925.0,...,82425,True,True,3.543100,True,True,False,True,0,1
600,2024-04-12 09:15:00,1339.05,1339.05,1330.60,1334.45,155400,6912000,VOLTAS,2024-04-12,-524400.0,...,155400,True,True,2.570002,True,True,False,True,0,0
669,2024-04-12 15:00:00,1320.95,1321.15,1311.20,1315.20,168000,7314600,VOLTAS,2024-04-12,-1200.0,...,168000,True,True,2.867631,True,True,False,True,0,0
670,2024-04-12 15:05:00,1315.20,1318.05,1312.10,1313.70,153000,7349400,VOLTAS,2024-04-12,34800.0,...,153000,True,True,2.513311,True,True,False,True,0,0


In [ ]:
# significant_changes = df_titan[df_titan['majority_flag']]
# significant_changes = significant_changes[significant_changes['date'].dt.date == pd.to_datetime('2024-04-12').date()]
# significant_changes = significant_changes[significant_changes['date'].dt.time.between(pd.to_datetime('09:15').time(), pd.to_datetime('10:00').time())]

## stock 9:25

1. 9:20 top gainer/ loser (Futures)
2. more then 2% move 
3. nse spurtes OI -> more then 7%
4. nifty 10 min candle  -> check 

5. entry:
	1. 9:25 mark high, if 9:25 high break, enter the trade
	 cond:
		dont trade if 50% pull back 
		enter before 10:30 
		monthly expire -> 2 din phale and 2 din bad 
		
	exit:
	1. 2 candle below 8 sma 
	
other condition
1. 9 day high or low 
2. OL or OH
3. day future some good buildup 
4. prev 3 or 4 day fut
5. pcr cutoff for above and below 
i have observed about PCR if its above 1 then most probably its uptrend 
and if its below .5 then down trend 



#can we check the pcr value and do the fundatmental analysis 


In [5]:
# futures some high OI
# last few days oi buildup 
# 9 day high
# OL or OH

In [10]:
from tqdm import tqdm

def fetch_market_data(kite, expiry_march, start_dt, end_dt, time_frame="minute"):
    df_expiry_hist_data = pd.DataFrame()
    strike_symbol_dict = dict(
        zip(expiry_march.instrument_token, expiry_march["name"])
    )
    inst_expiry = expiry_march["instrument_token"].unique().tolist()
    for i in tqdm(inst_expiry):
        try:
            df = pd.DataFrame(
                kite.historical_data(
                    i,
                    from_date=start_dt,
                    to_date=end_dt,
                    interval=time_frame,
                    continuous=False,
                    oi=True,
                )
            )
            df["symbol"] = strike_symbol_dict[i]
            df_expiry_hist_data = df_expiry_hist_data.append(df)
        except Exception as e:
            print(e)
            continue
    df_expiry_hist_data["date"] = pd.to_datetime(
        df_expiry_hist_data["date"]
    ).dt.tz_localize(None)
    df_expiry_hist_data["date_only"] = pd.to_datetime(
        df_expiry_hist_data["date"].dt.date
    )
    return df_expiry_hist_data

In [11]:
# select the stocks list which are in futures only
inst = pd.DataFrame(kite.instruments("NFO"))
inst_nse = pd.DataFrame(kite.instruments("NSE"))

april_fut_inst = inst[inst["tradingsymbol"].str.contains('24JULFUT')]


In [12]:

start_dt = "2024-07-01"
end_dt = "2024-07-25" 
time_frame = "day"

start_dt_td = "2024-07-01"
end_dt_td = "2024-07-25" 
time_frame_td = "5minute"

df_day_fut = fetch_market_data(kite,april_fut_inst, start_dt, end_dt,time_frame)
df_data_min_fut = fetch_market_data(kite,april_fut_inst, start_dt_td, end_dt_td,time_frame_td)

inst_nse = pd.DataFrame(kite.instruments("NSE"))
stock_token = inst_nse[inst_nse.tradingsymbol.isin(df_day_fut.symbol.unique())]

df_day = fetch_market_data(kite,stock_token, start_dt, end_dt, time_frame)
df_data_min = fetch_market_data(kite,stock_token, start_dt_td, end_dt_td,time_frame_td)

100%|██████████| 181/181 [00:54<00:00,  3.32it/s]


In [13]:
# getting fut oi % chg at 9:20 
list_of_dates_fut = list(df_data_min_fut["date_only"].unique())

df_data_min_fut["prev_date"] = df_data_min_fut["date_only"].apply(
    lambda x: (
        np.nan
        if list_of_dates_fut.index(x) == 0
        else list_of_dates_fut[list_of_dates_fut.index(x) - 1]
    )
)

df_data_min_fut = df_data_min_fut.merge(
    df_day_fut[["date_only", "symbol", "oi"]],
    how="left",
    left_on=["symbol", "prev_date"],
    right_on=["symbol", "date_only"],
    suffixes=("_min", "_day"),
)


df_data_min_fut["per_chg"] = (
    (df_data_min_fut["oi_min"] - df_data_min_fut["oi_day"]) / df_data_min_fut["oi_day"]
) * 100

df_data_min_fut.set_index("date", inplace=True)
df_oi_fut = df_data_min_fut.between_time("9:20",'9:20')#.dropna()


In [14]:
# getting price % chg at 9:20 
list_of_dates = list(df_data_min["date_only"].unique())

df_data_min["prev_date"] = df_data_min["date_only"].apply(
    lambda x: (
        np.nan
        if list_of_dates.index(x) == 0
        else list_of_dates[list_of_dates.index(x) - 1]
    )
)

df_data_min = df_data_min.merge(
    df_day[["date_only", "symbol", "close"]],
    how="left",
    left_on=["symbol", "prev_date"],
    right_on=["symbol", "date_only"],
    suffixes=("_min", "_day"),
)

df_data_min["per_chg"] = (
    (df_data_min["close_min"] - df_data_min["close_day"]) / df_data_min["close_day"]
) * 100

df_data_min.set_index("date", inplace=True)
df_nine_filt = df_data_min.between_time("9:20",'9:20')#.dropna()
df_nine_filt.reset_index(inplace=True)
df_nine_filt = df_nine_filt.merge(inst_nse[['tradingsymbol','name']], how ='left', left_on = 'symbol',right_on = 'name')
df_nine_filt = df_nine_filt.drop(['name','symbol'],axis =1 ).rename(columns= {'tradingsymbol':'symbol'})

In [15]:
(
    df_nine_filt
    .merge(df_oi_fut.reset_index()[['date','symbol','per_chg']],
            how = 'left',
            on = ['date','symbol'],
            suffixes = ("_price","_oi")
              
              )

).to_clipboard()

In [19]:
df_nine_filt[(df_nine_filt.per_chg > 2 )|( df_nine_filt.per_chg < -2)].to_clipboard()

# demand and supply zone 

In [1]:
import pandas as pd 
from tqdm import tqdm
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from kite_trade import *
from enctoken import get_kite
kite = get_kite()
import warnings
warnings.filterwarnings(action = 'ignore')
from IPython.display import clear_output
import datetime
pd.options.display.float_format = '{:.2f}'.format


In [2]:
def fetch_market_data(kite, expiry_march, start_dt, end_dt, time_frame="minute"):
    df_expiry_hist_data = pd.DataFrame()
    strike_symbol_dict = dict(
        zip(expiry_march.instrument_token, expiry_march["name"])
    )
    inst_expiry = expiry_march["instrument_token"].unique().tolist()
    for i in tqdm(inst_expiry):
        try:
            df = pd.DataFrame(
                kite.historical_data(
                    i,
                    from_date=start_dt,
                    to_date=end_dt,
                    interval=time_frame,
                    continuous=False,
                    oi=True,
                )
            )
            df["symbol"] = strike_symbol_dict[i]
            df_expiry_hist_data = df_expiry_hist_data.append(df)
        except Exception as e:
            print(e)
            continue
    df_expiry_hist_data["date"] = pd.to_datetime(
        df_expiry_hist_data["date"]
    ).dt.tz_localize(None)
    df_expiry_hist_data["date_only"] = pd.to_datetime(
        df_expiry_hist_data["date"].dt.date
    )
    return df_expiry_hist_data

In [3]:
df_nse = pd.read_csv("../data/ind_nifty500list.csv")

In [4]:
df_nse.head()

,Company Name,Industry,Symbol,Series,ISIN Code
0,3M India Ltd.,Diversified,3MINDIA,EQ,INE470A01017
1,ABB India Ltd.,Capital Goods,ABB,EQ,INE117A01022
2,ACC Ltd.,Construction Materials,ACC,EQ,INE012A01025
3,AIA Engineering Ltd.,Capital Goods,AIAENG,EQ,INE212H01026
4,APL Apollo Tubes Ltd.,Capital Goods,APLAPOLLO,EQ,INE702C01027


In [5]:
# select the stocks list which are in futures only
inst = pd.DataFrame(kite.instruments("NFO"))
inst_nse = pd.DataFrame(kite.instruments("NSE"))

fut_inst = inst[inst["tradingsymbol"].str.contains('4DECFUT')]

In [6]:
top_five_hun = inst_nse[inst_nse.tradingsymbol.isin(df_nse.Symbol.unique())]

In [7]:
# start_dt = "2020-01-01"
# end_dt = "2024-12-31" 
# time_frame = "day"

# # start_dt_td = "2024-07-01"
# # end_dt_td = "2024-07-25" 
# # time_frame_td = "5minute"


# df_day = fetch_market_data(kite,top_five_hun, start_dt, end_dt, time_frame)
# # df_data_min = fetch_market_data(kite,stock_token, start_dt_td, end_dt_td,time_frame_td)

# df_day.to_parquet("../data/day_level_2020_2024_12_31.parquet")
df_day = pd.read_parquet("../data/day_level_2020_2024_12_31.parquet")

In [120]:
# df_day[df_day.symbol.str.contains("VEDANTA")]

In [8]:
# Assuming 'data' is your dataframe
df_day['month'] = df_day['date'].dt.to_period('Q')  # Extract year-month

# Aggregate the data at the month level
monthly_data = df_day.groupby(['month', 'symbol']).agg(
    open=('open', 'first'),
    high=('high', 'max'),
    low=('low', 'min'),
    close=('close', 'last'),
    volume=('volume', 'sum'),
    oi=('oi', 'sum')
).reset_index()

# Convert month back to datetime if needed
monthly_data['month'] = monthly_data['month'].dt.to_timestamp()

# Calculate body size, total size, and body percentage
monthly_data['body_size'] = abs(monthly_data['close'] - monthly_data['open'])
monthly_data['total_size'] = monthly_data['high'] - monthly_data['low']
monthly_data['body_percentage'] = (monthly_data['body_size'] / monthly_data['total_size']) * 100

# Classify candles
def classify_candle(row):
    if row['body_percentage'] < 50:
        return 'Base'
    elif row['close'] > row['open']:
        return 'Rally'
    elif row['close'] < row['open']:
        return 'Drop'
    else:
        return 'Unknown'

monthly_data['candle_type'] = monthly_data.apply(classify_candle, axis=1)
monthly_data['candle_type'] = monthly_data['candle_type'].str.lower()

In [9]:
monthly_data['perc_change'] = abs((monthly_data['close'] - monthly_data['open']) / monthly_data['open']) * 100

In [10]:
monthly_data[monthly_data.symbol.str.contains("VEDANTA")].head()

,month,symbol,open,high,low,close,volume,oi,body_size,total_size,body_percentage,candle_type,perc_change
407,2020-01-01,VEDANTA,110.80,124.10,18.20,22.70,1281930251,0,88.10,105.90,83.19,drop,79.51
831,2020-04-01,VEDANTA,23.00,71.50,20.20,64.40,2078100869,0,41.40,51.30,80.70,rally,180.00
1259,2020-07-01,VEDANTA,64.40,99.50,61.65,94.90,1255655310,0,30.50,37.85,80.58,rally,47.36
1694,2020-10-01,VEDANTA,95.60,128.70,49.20,119.40,2761816951,0,23.80,79.50,29.94,base,24.90
2140,2021-01-01,VEDANTA,119.80,189.70,118.00,186.75,1137807120,0,66.95,71.70,93.38,rally,55.88


## pattern detect

1. remove the tested zone condition 
2. use base candle to check if the price is in range
3. add multiple base candle support 

In [124]:
# I have a dataframe which contains month	 symbol	open	high	low	close	volume	oi	body_size	total_size	body_percentage	candle_type

# and candle type is mark as  rally, base , drop, 

# Now i want to find those pattern which are 
# 1. drop base drop  as SZ
# 2. drop base rally as DZ
# 3. rally base rally as DZ
# 4. rally base drop as SZ


# Conditions:
# 1. Base candle can be 1 to 3 
# 2. Also flag those which zone are already tested, tested means price already came to the base candle 
# 3. Exiting candle should have condition that it should close above for green and below base candle 
# for red 

# i want to save months which create these pattern 

# skipping pattern which has alrady candle foramtion  - done 

# even if the candle is 50% body still the size is small as compared to neighbour candle -> mark those zone also

## code 0

In [11]:

def find_supply_demand_zones(df):
    """
    Scans through the DataFrame and identifies the following patterns:
      1. drop-base-drop  -> SZ
      2. drop-base-rally -> DZ
      3. rally-base-rally -> DZ
      4. rally-base-drop  -> SZ
    
    Returns a list of dictionaries with information about each found zone:
      - pattern_type: 'SZ' or 'DZ'
      - pattern_seq: e.g. 'drop-base-drop'
      - start_idx, end_idx: indices where pattern occurs
      - months: list of unique months across those candles
      - base_high, base_low: the high/low of the base region
      - tested: whether price has revisited the base
    """
    zones = []
    n = len(df)
    
    # Helper function to get the overall High/Low of the base region
    def get_base_high_low(df_slice):
        """
        Given a subset of rows that are 'base' candles,
        returns (max_high, min_low) across those base candles.
        """
        return df_slice['high'].max(), df_slice['low'].min()
    
    i = 0
    while i < n:
        first_candle_type = df.loc[i, 'candle_type']
        
        # We only proceed if the first candle is drop/rally
        if first_candle_type in ['drop','rally']:
            
            # Try 1 to 3 base candles
            for base_count in [1,2,3]:
                if i + 1 + base_count >= n:
                    break  # Not enough candles left to form a pattern
                
                # Slice out the potential base candles
                base_slice = df.iloc[i+1 : i+1+base_count]
                
                # Check if they're all 'base'
                if all(base_slice['candle_type'] == 'base'):
                    exit_idx = i + 1 + base_count
                    if exit_idx < n:
                        last_candle_type = df.loc[exit_idx, 'candle_type']
                        
                        # Combine them to see if we match one of the 4 patterns
                        possible_pattern = first_candle_type + '-base-' + last_candle_type
                        if possible_pattern in [
                            'drop-base-drop', 
                            'drop-base-rally', 
                            'rally-base-rally', 
                            'rally-base-drop'
                        ]:
                            # Check exit candle conditions
                            base_high, base_low = get_base_high_low(base_slice)
                            exit_close = df.loc[exit_idx, 'close']
                            
                            meets_exit_condition = False
                            if last_candle_type == 'rally':
                                # Must close > base_high
                                if exit_close > base_high:
                                    meets_exit_condition = True
                            elif last_candle_type == 'drop':
                                # Must close < base_low
                                if exit_close < base_low:
                                    meets_exit_condition = True
                            
                            if meets_exit_condition:
                                # Map pattern to zone type
                                #  'drop-base-drop'  -> SZ
                                #  'rally-base-drop' -> SZ
                                #  'drop-base-rally' -> DZ
                                #  'rally-base-rally'-> DZ
                                if possible_pattern in ['drop-base-drop', 'rally-base-drop']:
                                    zone_type = 'SZ'
                                else:
                                    zone_type = 'DZ'
                                
                                # Pattern found! Check if zone is tested after exit_idx
                                tested = is_zone_tested(df, exit_idx, base_high, base_low)
                                
                                # Gather months for the pattern range
                                pattern_start_idx = i
                                pattern_end_idx   = exit_idx
                                months_in_pattern = df.loc[pattern_start_idx:pattern_end_idx, 'month'].unique()
                                
                                zones.append({
                                    'pattern_type': zone_type,
                                    'pattern_seq': possible_pattern,
                                    'start_idx': pattern_start_idx,
                                    'end_idx': pattern_end_idx,
                                    'months': list(months_in_pattern),
                                    'base_high': base_high,
                                    'base_low': base_low,
                                    'tested': tested
                                })
        
        # IMPORTANT: We only move forward by 1 (so that the exit candle can be re-used)
        i += 1
    
    return zones

def is_zone_tested(df, pattern_end_idx, base_low, base_high):
    """
    Returns True if, AFTER 'pattern_end_idx', any candle's price range
    touches the [base_low, base_high] region.

    This strictly ignores the candles that formed the zone itself,
    meaning we only start checking from 'pattern_end_idx + 1'.
    """
    if pattern_end_idx >= len(df) - 1:
        return False  # no future candles to test
    
    # Look only at candles after the zone's exit candle
    future_slice = df.iloc[pattern_end_idx+1:]
    
    for idx, row in future_slice.iterrows():
        # If the candle's range intersects the base range,
        # we consider the zone tested.
        if (row['low'] <= base_high) and (row['high'] >= base_low):
            return True
    
    return False

# main

In [12]:
pattern_matches_all = pd.DataFrame()
for sym in tqdm(monthly_data.symbol.unique()):
# Apply the function
    # if sym == 'ABB INDIA':
        # print(sym)
        df_temp = monthly_data[monthly_data.symbol == sym].reset_index()
        # print(df_temp.shape)
        pattern_matches = find_supply_demand_zones(df_temp)
        pattern_matches = pd.DataFrame(pattern_matches)
        pattern_matches['symbol'] = sym
        pattern_matches_all = pd.concat([pattern_matches_all,pattern_matches])

pattern_matches = pd.concat([pattern_matches_all,pattern_matches_all.months.apply(pd.Series)],axis = 1)


100%|██████████| 477/477 [00:02<00:00, 183.58it/s]


In [13]:
df_temp

,index,month,symbol,open,high,low,close,volume,oi,body_size,total_size,body_percentage,candle_type,perc_change
0,4355,2022-04-01,RAINBOW CHILDRENS MED,510.00,517.90,410.00,465.95,35891863,0,44.05,107.90,40.82,base,8.64
1,4832,2022-07-01,RAINBOW CHILDRENS MED,465.95,672.00,457.15,630.85,16994571,0,164.90,214.85,76.75,rally,35.39
2,5309,2022-10-01,RAINBOW CHILDRENS MED,630.00,886.45,619.65,739.20,36436607,0,109.20,266.80,40.93,base,17.33
3,5786,2023-01-01,RAINBOW CHILDRENS MED,748.90,793.95,695.70,730.50,7927679,0,18.40,98.25,18.73,base,2.46
4,6263,2023-04-01,RAINBOW CHILDRENS MED,730.45,1008.30,725.00,961.10,12332450,0,230.65,283.30,81.42,rally,31.58
5,6740,2023-07-01,RAINBOW CHILDRENS MED,969.90,1184.60,941.25,1028.35,20408524,0,58.45,243.35,24.02,base,6.03
6,7217,2023-10-01,RAINBOW CHILDRENS MED,1036.60,1215.90,992.95,1193.05,13330028,0,156.45,222.95,70.17,rally,15.09
7,7694,2024-01-01,RAINBOW CHILDRENS MED,1200.40,1487.65,1091.00,1297.55,15953987,0,97.15,396.65,24.49,base,8.09
8,8171,2024-04-01,RAINBOW CHILDRENS MED,1310.00,1649.00,1204.00,1245.85,13201214,0,64.15,445.00,14.42,base,4.90
9,8648,2024-07-01,RAINBOW CHILDRENS MED,1255.00,1518.70,1079.10,1399.85,16152922,0,144.85,439.60,32.95,base,11.54


In [14]:
pattern_matches.to_clipboard(index = False)

### code 1

In [111]:
def detect_patterns(group):
    # Create shifted columns to form patterns
    group['prev_1'] = group['candle_type'].shift(1)
    group['prev_2'] = group['candle_type'].shift(2)

    group['prev_date_1'] = group['month'].shift(1)
    group['prev_date_2'] = group['month'].shift(2)
    
    # Combine the previous two rows with the current row into a pattern
    group['pattern'] = (
        group['prev_2'] + ' ' +
        group['prev_1'] + ' ' +
        group['candle_type']
    )
    
    # Define patterns to search
    patterns = {
        'drop-base-drop': 'Drop Base Drop',
        'drop-base-rally': 'Drop Base Rally',
        'rally-base-drop': 'Rally Base Drop',
        'rally-base-rally': 'Rally Base Rally'
    }
    
    # Map detected patterns to their names
    group['pattern_type'] = group['pattern'].map({v: k for k, v in patterns.items()})
    
    # Drop rows where patterns cannot be formed
    group = group.dropna(subset=['pattern_type'])
    
    return group

# Apply the function to each symbol group
pattern_matches = monthly_data.groupby('symbol', group_keys=False).apply(detect_patterns)

### code 2

In [114]:
# Function to find patterns
def find_patterns(df):
    patterns = []
    tested_zones = []

    for i in range(len(df) - 2):
        # Check for the specific pattern
        if (
            df.loc[i, "candle_type"] in ["drop", "rally"]
            and df.loc[i + 1, "candle_type"] == "base"
            and df.loc[i + 2, "candle_type"] in ["drop", "rally"]
        ):
            patterns.append((i, i + 1, i + 2))  # Save pattern indices

            # Check if zone is tested
            base_low = df.loc[i + 1, "low"]
            base_high = df.loc[i + 1, "high"]
            tested = any(
                (base_low <= df.loc[j, "low"] <= base_high)
                or (base_low <= df.loc[j, "high"] <= base_high)
                for j in range(i + 3, len(df))
            )
            if tested == False:
                print(df.loc[i,:],df.loc[i+1,:],df.loc[i+2,:])
                print()
                print()
            tested_zones.append(tested)

    return patterns, tested_zones


monthly_data['candle_type'] = monthly_data['candle_type'].str.lower()
df_pattern_matches = pd.DataFrame()
for sym in tqdm(monthly_data.symbol.unique()):
# Apply the function
    if sym == 'SUNTV':
        df_temp = monthly_data[monthly_data.symbol == sym].reset_index()
        patterns, tested_zones = find_patterns(df_temp)

        # Flag the patterns and tested zones in the dataframe
        df_temp["pattern_flag"] = ""
        df_temp["tested_zone"] = False

        for idx, pattern in enumerate(patterns):
            p_start, base_idx, p_end = pattern
            pattern_type = f"{df_temp.loc[p_start, 'candle_type']}-base-{df_temp.loc[p_end, 'candle_type']}"
            df_temp.loc[[p_start, base_idx, p_end], "pattern_flag"] = pattern_type
            df_temp.loc[base_idx, "tested_zone"] = tested_zones[idx]
        df_pattern_matches = pd.concat([df_pattern_matches, df_temp],ignore_index=True)
    # break

100%|██████████| 477/477 [00:00<00:00, 1505404.82it/s]

In [91]:
pattern_matches = df_pattern_matches.query("pattern_flag != '' and tested_zone == False")

## code 3

In [69]:
import pandas as pd

# ------------------------------------------------------------
# 1) Helper function to check if the exit candle is valid
# ------------------------------------------------------------
def is_valid_exit_candle(candle, exit_type, base_candles):
    """
    Checks if 'candle' (a row from the DataFrame) is a valid exit candle 
    given 'exit_type' ('rally' or 'drop') and the 'base_candles' (DataFrame slice).
    
    Conditions:
      - For 'rally': close > max_base_high AND body_size > 1.5 * avg_base_body
      - For 'drop':  close < min_base_low  AND body_size > 1.5 * avg_base_body
    """
    max_base_high = base_candles['high'].max()
    min_base_low  = base_candles['low'].min()
    avg_base_body = base_candles['body_size'].mean()

    if exit_type == 'rally':
        cond1 = candle['close'] > max_base_high
        cond2 = candle['body_size'] > 1.5 * avg_base_body
        return cond1 and cond2

    elif exit_type == 'drop':
        cond1 = candle['close'] < min_base_low
        cond2 = candle['body_size'] > 1.5 * avg_base_body
        return cond1 and cond2

    return False


# ------------------------------------------------------------
# 2) Helper function to check if a zone is already tested
# ------------------------------------------------------------
def is_zone_tested(df, start_idx, base_zone_low, base_zone_high, is_demand=True):
    """
    Checks if the zone [base_zone_low, base_zone_high] is already tested 
    by any candle from 'start_idx' onward in the DataFrame 'df'.
    
    - If is_demand=True (like 'drop base rally' or 'rally base rally'), 
      then we check if any candle's low has entered this zone.
    - If is_demand=False (like 'rally base drop' or 'drop base drop'), 
      then we check if any candle's high has entered this zone.
    """
    for j in range(start_idx, len(df)):
        if is_demand:
            # For a demand zone: if the candle's low is within [low, high], it's tested
            if base_zone_low <= df.iloc[j]['low'] <= base_zone_high:
                return True
        else:
            # For a supply zone: if the candle's high is within [low, high], it's tested
            if base_zone_low <= df.iloc[j]['high'] <= base_zone_high:
                return True
    return False


# ------------------------------------------------------------
# 3) Main function to find supply/demand patterns in the DataFrame
# ------------------------------------------------------------
def find_supply_demand_patterns(df):
    """
    Scans the DataFrame 'df' to find these patterns:
      1) drop base drop
      2) drop base rally
      3) rally base drop
      4) rally base rally

    Rules:
      - 'base' can be 1-3 consecutive candles labeled as 'base'.
      - The exit candle must:
          - close above the base's highest high (for rally) or below the base's lowest low (for drop)
          - have a body_size > 1.5 x average body_size of base candles
      - A zone is flagged as 'tested' if subsequent candles come back into the base range:
          - For demand zones (e.g. drop base rally, rally base rally): check candle low
          - For supply zones (e.g. rally base drop, drop base drop): check candle high

    Returns a DataFrame with found patterns and additional info.
    """

    patterns = []
    n = len(df)
    i = 0

    while i < n - 1:
        first_candle_type = df.iloc[i]['candle_type']

        # We only look for 'drop' or 'rally' as the first candle in the pattern
        if first_candle_type not in ['drop', 'rally']:
            i += 1
            continue

        # Try to find 1-3 consecutive base candles after the first candle
        base_start = i + 1
        base_end_candidates = []

        for base_count in [1, 2, 3]:
            # If slicing goes out of range, break
            if base_start + base_count - 1 >= n:
                break
            # Slicing these 'base_count' candles
            potential_base = df.iloc[base_start : base_start + base_count]
            # Check if they are all 'base'
            if all(potential_base['candle_type'] == 'base'):
                # This is a valid base block
                base_end_candidates.append(base_start + base_count - 1)

        # For each valid base block, check the next candle (exit candle)
        for base_end in base_end_candidates:
            exit_idx = base_end + 1
            if exit_idx >= n:
                break

            exit_candle_type = df.iloc[exit_idx]['candle_type']

            # Construct the potential pattern name: e.g. "drop base drop"
            potential_pattern_name = f"{first_candle_type} base {exit_candle_type}"

            # The base candles slice
            base_candles = df.iloc[base_start : base_end + 1]

            if exit_candle_type in ['drop', 'rally']:
                # Check if the exit candle meets the exit criteria
                exit_candle_row = df.iloc[exit_idx]
                if is_valid_exit_candle(exit_candle_row, exit_candle_type, base_candles):
                    # We have found a valid pattern
                    base_zone_low  = base_candles['low'].min()
                    base_zone_high = base_candles['high'].max()

                    # Determine if this zone is a demand or supply zone
                    # For simplicity:
                    #   - "drop base rally"  => demand zone
                    #   - "rally base rally" => demand zone
                    #   - "rally base drop"  => supply zone
                    #   - "drop base drop"   => supply zone
                    if potential_pattern_name in ['drop base rally', 'rally base rally']:
                        tested = is_zone_tested(df, exit_idx + 1, base_zone_low, base_zone_high, is_demand=True)
                    else:
                        tested = is_zone_tested(df, exit_idx + 1, base_zone_low, base_zone_high, is_demand=False)

                    # Append the pattern info
                    patterns.append({
                        'start_idx': i,
                        'base_idx_range': (base_start, base_end),
                        'exit_idx': exit_idx,
                        'pattern_name': potential_pattern_name,
                        'base_zone_low': base_zone_low,
                        'base_zone_high': base_zone_high,
                        'tested': tested
                    })

        i += 1

    # Return the patterns as a DataFrame for convenience
    return pd.DataFrame(patterns)


# ------------------------------------------------------------
# 4) Example usage with your DataFrame: 'monthly_data'
# ------------------------------------------------------------
# Assume 'monthly_data' already has these columns:
#   ['month','symbol','open','high','low','close','volume','oi',
#    'body_size','total_size','body_percentage','candle_type']



# Now 'patterns_df' will contain rows for each detected pattern with columns:
#   ['start_idx','base_idx_range','exit_idx','pattern_name',
#    'base_zone_low','base_zone_high','tested']

In [70]:

for sym in tqdm(monthly_data.symbol.unique()):
# Apply the function
    if sym == 'SUN TV NETWORK':
        print(sym)
        df_temp = monthly_data[monthly_data.symbol == sym].reset_index()
        print(df_temp.shape)
        pattern_matches = find_supply_demand_patterns(df_temp)

100%|██████████| 477/477 [00:00<00:00, 18343.45it/s]

SUN TV NETWORK
(60, 13)


In [71]:
pattern_matches

,start_idx,base_idx_range,exit_idx,pattern_name,base_zone_low,base_zone_high,tested
0,3,"(4, 6)",7,rally base rally,355.50,427.00,True
1,7,"(8, 8)",9,rally base drop,439.30,508.25,True
2,15,"(16, 17)",18,rally base rally,487.00,571.00,True
3,19,"(20, 20)",21,drop base rally,478.60,534.00,True
4,39,"(40, 41)",42,rally base rally,423.00,467.40,False
5,43,"(44, 45)",46,rally base rally,577.45,666.50,True
6,49,"(50, 50)",51,drop base rally,567.60,648.70,True
7,51,"(52, 52)",53,rally base rally,625.50,681.20,False
8,55,"(56, 56)",57,drop base drop,782.35,853.70,True
9,57,"(58, 58)",59,drop base drop,702.35,773.30,False


### code 4

In [28]:
def find_snd_patterns(df):
    """
    Identify supply-demand type patterns in the given DataFrame df.
    Looks for:
      1. drop-base-drop
      2. drop-base-rally
      3. rally-base-drop
      4. rally-base-rally

    Conditions:
      - 1 to 3 consecutive base candles
      - Exit candle must close above(below) the base for rally(drop)
      - Exit candle body size >= 1.5 x average body size of the base candles
    Returns:
      A list of dictionaries, where each dict contains:
         {
           'start_idx': index of the first move candle (drop/rally),
           'base_idxs': list of indexes for the base candles,
           'end_idx': index of the exit candle (drop/rally),
           'pattern': one of the four patterns,
           'tested': True/False if the zone has been tested in future candles
         }
    """
    patterns = []
    n = len(df)

    i = 0
    while i < n:
        first_type = df.loc[i, 'candle_type']
        
        # We only consider a pattern start if it's either 'drop' or 'rally'
        if first_type not in ['drop', 'rally']:
            i += 1
            continue
        
        # Try base lengths of 1, 2, or 3
        found_pattern = False
        for base_count in [1, 2, 3]:
            if i + base_count >= n:
                break
            
            # Check if the next base_count candles are all 'base'
            base_indices = list(range(i+1, i+1+base_count))
            
            # Ensure we have enough rows
            if max(base_indices) >= n:
                break
            
            if not all(df.loc[idx, 'candle_type'] == 'base' for idx in base_indices):
                # if *any* candle in this range is not base, move on
                continue
            
            # The exit candle is the one following the base candles
            exit_idx = i + 1 + base_count
            if exit_idx >= n:
                break
            
            exit_type = df.loc[exit_idx, 'candle_type']
            
            # Construct the pattern label, if valid
            pattern_label = None
            if first_type == 'drop' and exit_type == 'drop':
                pattern_label = 'drop-base-drop'
            elif first_type == 'drop' and exit_type == 'rally':
                pattern_label = 'drop-base-rally'
            elif first_type == 'rally' and exit_type == 'drop':
                pattern_label = 'rally-base-drop'
            elif first_type == 'rally' and exit_type == 'rally':
                pattern_label = 'rally-base-rally'
            
            if pattern_label is None:
                # Not one of our four desired patterns
                continue
            
            # 1) Check the body-size requirement for the exit candle
            base_body_sizes = df.loc[base_indices, 'body_size']
            avg_base_body_size = base_body_sizes.mean()
            exit_body_size = df.loc[exit_idx, 'body_size']
            
            if exit_body_size < 1 * avg_base_body_size:
                # Fails the "1.5x" rule
                continue
            
            # 2) Check close above(below) base region
            base_high = df.loc[base_indices, 'high'].max()
            base_low = df.loc[base_indices, 'low'].min()
            exit_close = df.loc[exit_idx, 'close']
            
            # Rally must close above base_high; drop must close below base_low
            if (exit_type == 'rally' and exit_close <= base_high):
                continue
            if (exit_type == 'drop' and exit_close >= base_low):
                continue
            
            # If we get here, we've found a valid pattern
            found_pattern = True
            
            # Mark whether the base zone is tested by future candles
            tested_flag = False
            
            # We’ll consider any future candle that intersects [base_low, base_high]
            # as a test of the zone.
            for future_idx in range(exit_idx + 1, n):
                f_low = df.loc[future_idx, 'low']
                f_high = df.loc[future_idx, 'high']
                # If the candle range intersects the base zone, consider it tested.
                if (f_low <= base_high) and (f_high >= base_low):
                    tested_flag = True
                    break
            
            patterns.append({
                'start_idx': i,
                'base_idxs': base_indices,
                'end_idx': exit_idx,
                'pattern': pattern_label,
                'tested': tested_flag
            })
            
            # Since we found a valid pattern starting at i, 
            # we can break out of the base_count loop. 
            # Or you might choose to keep scanning for other possible 
            # base_count lengths at the same i (if desired).
            break

        i += 1  # move forward

    return patterns

In [29]:

for sym in tqdm(monthly_data.symbol.unique()):
# Apply the function
    if sym == 'SUN TV NETWORK':
        print(sym)
        df_temp = monthly_data[monthly_data.symbol == sym].reset_index()
        print(df_temp.shape)
        pattern_matches = find_snd_patterns(df_temp)

100%|██████████| 477/477 [00:00<00:00, 136082.37it/s]

SUN TV NETWORK
(60, 13)


In [30]:
pattern_matches

[]

### code 5

In [ ]:
def is_fresh_zone(df, start_idx, end_idx, base_low, base_high):
    """
    Check that for all candles after end_idx, 
    the high is not below base_low for demand zones, 
    or the low is not above base_high for supply zones,
    or more generally, that the zone is never penetrated.
    
    In simple terms: ensure that the subsequent candles' 
    range does not overlap with [base_low, base_high].
    """
    subsequent = df.iloc[end_idx + 1 :]
    for i, row in subsequent.iterrows():
        # If the candle's low is < base_high and the candle's high is > base_low,
        # that means it touched the zone [base_low, base_high].
        # If we want zero penetration, check that the entire candle is either 
        # above base_high or below base_low.
        if not (row['high'] < base_low or row['low'] > base_high):
            # We have a penetration
            return False
    return True

In [47]:
import pandas as pd

def find_drop_base_drop_rally_patterns(monthly_data):
    patterns_found = []

    # We might want to group by symbol
    for symbol, df_symbol in monthly_data.groupby('symbol'):
        # print(symbol,df_symbol.shape)
        # Sort by month (assuming ascending chronological order)
        df_symbol = df_symbol.sort_values(by='month').reset_index(drop=True)
        
        # Convert to list (or just keep it as a DF) for iteration
        # We'll do index-based iteration so we can look ahead.
        n = len(df_symbol)
        
        i = 0
        while i < n - 2:
            current_candle_type = df_symbol.loc[i, 'candle_type']
            
            # We only proceed if current is drop or rally,
            # because the next pattern is "drop -> base -> x" or "rally -> base -> x"
            if current_candle_type not in ['drop', 'rally']:
                i += 1
                continue
            
            # Now let's see if the next 1 to 3 candles are all base
            # We'll try base_length in [1, 2, 3]
            found_base_sequence = False
            for base_length in [1]:
                # Check if we have enough candles
                if i + 1 + base_length >= n:
                    continue
                
                base_candles = df_symbol.iloc[i+1 : i+1+base_length]
                
                # Are they all 'base'?
                if all(base_candles['candle_type'] == 'base'):
                    # If yes, then the next candle after these bases is an exit candle
                    exit_idx = i + 1 + base_length
                    if exit_idx < n:
                        exit_candle_type = df_symbol.loc[exit_idx, 'candle_type']
                        
                        # Check if exit candle is 'drop' or 'rally'
                        if exit_candle_type in ['drop', 'rally']:
                            # We have a potential pattern:
                            #  current_candle_type -> base -> exit_candle_type
                            pattern = f"{current_candle_type} base {exit_candle_type}"
                            
                            # 1) Check exit condition (close above/below base)
                            base_low = base_candles['low'].min()
                            base_high = base_candles['high'].max()
                            exit_close = df_symbol.loc[exit_idx, 'close']
                            
                            # For rally exit, close must be > base_high
                            # For drop exit, close must be < base_low
                            if (exit_candle_type == 'rally' and exit_close > base_high) \
                               or (exit_candle_type == 'drop' and exit_close < base_low):
                                
                                # 2) Check fresh zone condition
                                if is_fresh_zone(
                                    df_symbol, 
                                    start_idx=i, 
                                    end_idx=exit_idx, 
                                    base_low=base_low, 
                                    base_high=base_high
                                ):
                                    # If fresh zone is True, we record the pattern
                                    patterns_found.append({
                                        'symbol': symbol,
                                        'start_index': i,
                                        'base_start_index': i + 1,
                                        'base_end_index': i + base_length,
                                        'exit_index': exit_idx,
                                        'pattern': pattern,
                                        'base_range': (base_low, base_high),
                                        'start_month': df_symbol.loc[i, 'month'],
                                        'exit_month': df_symbol.loc[exit_idx, 'month']
                                    })
                    
                    # We found a base sequence; break out so we don't double count
                    found_base_sequence = True
                    break
                
            # Move to next candle
            i += 1

    # Convert found patterns to DataFrame
    patterns_df = pd.DataFrame(patterns_found)
    return patterns_df

In [48]:
find_drop_base_drop_rally_patterns(monthly_data)

""


In [37]:

for sym in tqdm(monthly_data.symbol.unique()):
# Apply the function
    # if sym == 'SUN TV NETWORK':
        df_temp = monthly_data[monthly_data.symbol == sym].reset_index()
        pattern_matches = find_drop_base_drop_rally_patterns(df_temp)

  0%|          | 0/477 [00:00<?, ?it/s]

100%|██████████| 477/477 [00:00<00:00, 483.25it/s]


In [38]:
pattern_matches

""


### code 6

In [60]:
import pandas as pd

def find_variable_base_patterns(df):
    """
    Given a DataFrame (already sorted for one symbol),
    return a list of found patterns.
    """
    # We'll keep results in a list of dicts
    results = []
    
    i = 0
    n = len(df)
    while i < n:
        start_candle = df['candle_type'].iloc[i]
        print(start_candle)
        
        # 1) Check if the current candle is 'drop' or 'rally'
        if start_candle in ('drop', 'rally'):
            # 2) Look ahead up to 3 candles for consecutive bases
            base_count = 0
            j = i + 1
            while j < n and base_count < 3 and df['candle_type'].iloc[j] == 'base':
                base_count += 1
                j += 1
            
            # base_count is how many base candles we found (could be 0 to 3)
            # We only care if base_count >= 1
            if 1 <= base_count <= 3:
                # 3) After these base candles, check the next candle
                if j < n:
                    end_candle = df['candle_type'].iloc[j]
                    if end_candle in ('drop', 'rally'):
                        # We found a valid pattern
                        # Example label: 'drop_base_drop', 'drop_base_base_drop', ...
                        # For readability, let's build a pattern string that 
                        # literally includes as many 'base' words as we have:
                        pattern = (
                            f"{start_candle}" 
                            + "".join(["_base" for _ in range(base_count)]) 
                            + f"_{end_candle}"
                        )
                        
                        # Store some info—row indices, pattern, etc.
                        # The pattern covers rows [i, i+1..j-1, j]
                        # i = start, i+1..j-1 = base(s), j = end
                        results.append({
                            'symbol': df['symbol'].iloc[i],
                            'start_idx': df.index[i],
                            'base_indices': df.index[i+1:j],  
                            'end_idx': df.index[j],
                            'pattern': pattern
                        })
        i += 1  # move on to the next row
    
    return results



In [ ]:

monthly_data.sort_values(by=['symbol', 'month'], inplace=True)
monthly_data.candle_type = monthly_data.candle_type.str.lower()
# We'll build a list of all pattern matches across all symbols
all_patterns = []

# groupby symbol, then apply
for symbol, gdf in monthly_data.groupby('symbol'):
    print(symbol)
    if symbol == "ACC":
        # gdf is sorted for each symbol, so we can pass it to our function
        symbol_patterns = find_variable_base_patterns(gdf)
        all_patterns.extend(symbol_patterns)
        break
# Convert that list of dicts to a DataFrame if you wish:
patterns_df = pd.DataFrame(all_patterns)

# patterns_df now has one row per matched pattern. 
# The 'pattern' column might look like "drop_base_drop" or "rally_base_base_rally", etc.
# The 'start_idx', 'base_indices', 'end_idx' give you the exact candle rows involved.

In [59]:
patterns_df

""


In [50]:
patterns_df

""


## checking price in current price

In [99]:
# pattern_matches['date'] = pattern_matches['month']

In [15]:
start_dt = "2025-01-13"
end_dt = "2025-01-17" 
time_frame = "day"

# start_dt_td = "2024-07-01"
# end_dt_td = "2024-07-25" 
# time_frame_td = "5minute"


df_current = fetch_market_data(kite,top_five_hun, start_dt, end_dt, time_frame)

100%|██████████| 476/476 [00:29<00:00, 16.24it/s]


In [16]:
# Assuming 'data' is your dataframe
df_current['week'] = df_current['date'].dt.to_period('W')  # Extract year-month

# Aggregate the data at the month level
monthly_data_current = df_current.groupby(['week', 'symbol']).agg(
    open=('open', 'first'),
    high=('high', 'max'),
    low=('low', 'min'),
    close=('close', 'last'),
    volume=('volume', 'sum'),
    oi=('oi', 'sum')
).reset_index()

# Convert month back to datetime if needed
monthly_data_current['week'] = monthly_data_current['week'].dt.to_timestamp()

In [17]:
monthly_data_current

,week,symbol,open,high,low,close,volume,oi
0,2025-01-13,3M INDIA,29500.00,30750.00,28625.00,29700.00,47229,0
1,2025-01-13,AARTI DRUGS,429.05,431.95,413.15,421.95,641886,0
2,2025-01-13,AAVAS FINANCIERS,1656.00,1661.95,1625.00,1638.20,755404,0
3,2025-01-13,ABB INDIA,6352.00,6553.70,6061.05,6525.00,1451762,0
4,2025-01-13,ABBOTT INDIA,28613.00,28700.10,27102.45,27965.75,58299,0
...,...,...,...,...,...,...,...,...
471,2025-01-13,ZENSAR TECHNOLOGIES,779.00,791.90,716.70,750.00,3255953,0
472,2025-01-13,ZF COM VE CTR SYS IND,11265.00,11574.55,11027.05,11225.00,78634,0
473,2025-01-13,ZOMATO,240.00,262.00,226.20,248.65,329724107,0
474,2025-01-13,ZYDUS LIFESCIENCES,990.10,1007.30,966.25,992.70,4790019,0


In [18]:
pattern_matches.shape

(1313, 14)

In [19]:
pattern_matches = pattern_matches.merge(monthly_data_current, how = 'left', on = ['symbol'],suffixes=('_month','_day'))

In [20]:
pattern_matches.head()

,pattern_type,pattern_seq,start_idx,end_idx,months,base_high,base_low,tested,symbol,0,...,2,3,4,week,open,high,low,close,volume,oi
0,SZ,drop-base-drop,5.00,8.00,"[2021-04-01T00:00:00.000000000, 2021-07-01T00:...",27800.00,22575.25,True,3M INDIA,2021-04-01,...,2021-10-01,2022-01-01,NaT,2025-01-13,29500.00,30750.00,28625.00,29700.00,47229,0
1,DZ,drop-base-rally,8.00,10.00,"[2022-01-01T00:00:00.000000000, 2022-04-01T00:...",22857.90,17273.00,False,3M INDIA,2022-01-01,...,2022-07-01,NaT,NaT,2025-01-13,29500.00,30750.00,28625.00,29700.00,47229,0
2,DZ,drop-base-rally,11.00,13.00,"[2022-10-01T00:00:00.000000000, 2023-01-01T00:...",23994.95,21300.45,False,3M INDIA,2022-10-01,...,2023-04-01,NaT,NaT,2025-01-13,29500.00,30750.00,28625.00,29700.00,47229,0
3,SZ,rally-base-drop,17.00,19.00,"[2024-04-01T00:00:00.000000000, 2024-07-01T00:...",40856.50,33444.05,False,3M INDIA,2024-04-01,...,2024-10-01,NaT,NaT,2025-01-13,29500.00,30750.00,28625.00,29700.00,47229,0
4,SZ,rally-base-drop,2.00,6.00,"[2020-07-01T00:00:00.000000000, 2020-10-01T00:...",1026.95,615.65,False,AARTI DRUGS,2020-07-01,...,2021-01-01,2021-04-01,2021-07-01,2025-01-13,429.05,431.95,413.15,421.95,641886,0


In [21]:
def calculate_proximity(row):
    # Check if high and low are within base range
    high_in_range = row['base_low'] <= row['high'] <= row['base_high']
    low_in_range = row['base_low'] <= row['low'] <= row['base_high']
    
    # Calculate percentage difference from base range
    high_diff = abs(row['high'] - row['base_high']) / row['base_high'] * 100
    low_diff = abs(row['low'] - row['base_low']) / row['base_low'] * 100
    
    # Determine which value is closer to the base range
    if high_diff < low_diff:
        closer_value = 'high'
        closer_diff = high_diff
    else:
        closer_value = 'low'
        closer_diff = low_diff
    
    return pd.Series([high_in_range, low_in_range, closer_value, closer_diff],
                     index=['high_in_range', 'low_in_range', 'closer_value', 'closer_diff'])

# Apply function to DataFrame
pattern_matches[['high_in_range', 'low_in_range', 'closer_value', 'closer_diff']] = pattern_matches.apply(calculate_proximity, axis=1)


In [22]:
pattern_matches.to_clipboard()

In [87]:
pattern_matches['price_in_range'] = np.where(
    (pattern_matches['open_day'] >= pattern_matches['open_month']) & 
    (pattern_matches['open_day'] <= pattern_matches['close_month']) & 
    (pattern_matches['open_day'] >= pattern_matches['open_month']) & 
    (pattern_matches['open_day'] <= pattern_matches['close_month']), True, False
)

In [89]:
pattern_matches

,index,month,symbol,open_month,high_month,low_month,close_month,volume_month,oi_month,body_size,...,pattern_flag,tested_zone,week,open_day,high_day,low_day,close_day,volume_day,oi_day,price_in_range
0,427,2020-02-01,ADANI GREEN ENERGY,192.80,221.65,154.85,154.95,30169905,0,37.85,...,drop-base-rally,False,2024-12-23,1049.7,1059.05,1020.0,1031.05,5270895,0,False
1,848,2020-03-01,ADANI GREEN ENERGY,156.60,163.00,111.90,153.25,16710919,0,3.35,...,drop-base-rally,False,2024-12-23,1049.7,1059.05,1020.0,1031.05,5270895,0,False
2,1272,2020-04-01,ADANI GREEN ENERGY,155.50,212.85,150.45,210.45,8683611,0,54.95,...,drop-base-rally,False,2024-12-23,1049.7,1059.05,1020.0,1031.05,5270895,0,False
3,6892,2021-05-01,ADANI GREEN ENERGY,1015.00,1390.00,990.00,1268.20,18086585,0,253.20,...,rally-base-drop,False,2024-12-23,1049.7,1059.05,1020.0,1031.05,5270895,0,True
4,7789,2021-07-01,ADANI GREEN ENERGY,1124.90,1128.40,875.00,882.30,3756658,0,242.60,...,rally-base-drop,False,2024-12-23,1049.7,1059.05,1020.0,1031.05,5270895,0,False
5,21494,2023-12-01,ADANI GREEN ENERGY,1042.00,1700.00,1020.00,1597.00,87525688,0,555.00,...,rally-base-rally,False,2024-12-23,1049.7,1059.05,1020.0,1031.05,5270895,0,True
6,22448,2024-02-01,ADANI GREEN ENERGY,1689.00,2018.95,1651.60,1895.05,43456863,0,206.05,...,rally-base-rally,False,2024-12-23,1049.7,1059.05,1020.0,1031.05,5270895,0,False
7,26264,2024-10-01,ADANI GREEN ENERGY,1915.55,1929.55,1588.00,1598.30,14900493,0,317.25,...,drop-base-drop,False,2024-12-23,1049.7,1059.05,1020.0,1031.05,5270895,0,False
8,26741,2024-11-01,ADANI GREEN ENERGY,1620.00,1733.95,870.25,1323.90,117334950,0,296.10,...,drop-base-drop,False,2024-12-23,1049.7,1059.05,1020.0,1031.05,5270895,0,False
9,27218,2024-12-01,ADANI GREEN ENERGY,1360.55,1447.70,1015.10,1034.35,92751385,0,326.20,...,drop-base-drop,False,2024-12-23,1049.7,1059.05,1020.0,1031.05,5270895,0,False


In [79]:
pattern_matches[pattern_matches['price_in_range'] == True].to_clipboard()

In [ ]:
# exciting candle should be above average of candles 

# plots

In [76]:
import pandas as pd
import matplotlib.pyplot as plt
import calendar

# Load your Excel file
file_path = '/Users/akash_verma/Downloads/LEDGER REPORT29_12_2024.xlsx'
df = pd.read_excel(file_path, engine = 'openpyxl')
df.rename(columns = {"Doc Date":'Date'},inplace=True)
df.Date = pd.to_datetime(df.Date)

In [78]:
df.head()

,Date,Doc No,Cheque No,Narration,Dr/Cr,Amount,Balance,Year,Month,Day
0,NaT,NaN,nan,Projected Balance (C = B + A),NaN,nan,38729.01 Cr,nan,NaN,nan
1,NaT,NaN,nan,Future Dated Entries (B),NaN,nan,NaN,nan,NaN,nan
2,NaT,NaN,nan,Balance (A),NaN,nan,38729.01 Cr,nan,NaN,nan
3,2024-12-27,TJV2412271,186346.00,Bill for the trade date 27/12/2024 No - 186346,Cr,2728.69,38729.01 Cr,2024.00,December,27.00
4,2024-12-26,TJV2412261,185645.00,Bill for the trade date 26/12/2024 No - 185645,Cr,3923.33,36000.32 Cr,2024.00,December,26.00


In [74]:
import pandas as pd
import plotly.express as px
import numpy as np

# Load your data
# Assuming your data is in a CSV, replace 'your_data.csv' with the correct file path
# df = pd.read_csv('your_data.csv')
# Uncomment and use the following line to read your specific file if needed:
# df = pd.read_excel('path_to_your_file.xlsx')

# Ensure the 'Date' column is in datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Extract year, month, and day
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month_name()  # Get month names for better readability
df['Day'] = df['Date'].dt.day

# Aggregate data by date to calculate the total amount per day
daily_data = df.groupby(['Year', 'Month', 'Day'])['Amount'].sum().reset_index()

# Create a new column for sorting months chronologically
daily_data['Month_Num'] = pd.to_datetime(daily_data['Month'], format='%B').dt.month
daily_data = daily_data.sort_values(by=['Year', 'Month_Num', 'Day'])

# Create the interactive plot
fig = px.scatter(
    daily_data,
    x='Day',
    y='Month',
    # size='Amount',
    color='Amount',
    color_continuous_scale='RdYlGn',
    title='Interactive Day-wise Visualization of Amount',
    labels={'Day': 'Day of the Month', 'Month': 'Month'},
    hover_data={'Amount': ':.2f', 'Year': True}  # Format hover data
)

fig.update_layout(
    yaxis_title="Month",
    xaxis_title="Day of the Month",
    template="plotly_white",
    coloraxis_colorbar=dict(title="Amount"),
)

# Show the plot
fig.show()

In [77]:
import pandas as pd
import plotly.express as px
import numpy as np

# Load your data
# Assuming your data is in a CSV, replace 'your_data.csv' with the correct file path
# df = pd.read_csv('your_data.csv')
# Uncomment and use the following line to read your specific file if needed:
# df = pd.read_excel('path_to_your_file.xlsx')

# Ensure the 'Date' column is in datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Extract year, month, and day
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month_name()  # Get month names for better readability
df['Day'] = df['Date'].dt.day

# Aggregate data by date to calculate the total amount per day
daily_data = df.groupby(['Year', 'Month', 'Day'])['Amount'].sum().reset_index()

# Create a new column for sorting months chronologically
daily_data['Month_Num'] = pd.to_datetime(daily_data['Month'], format='%B').dt.month
daily_data = daily_data.sort_values(by=['Year', 'Month_Num', 'Day'])

# Define a custom color scale for positive (green) and negative (red) values
custom_color_scale = [
    [0.0, 'red'],    # For negative amounts
    [0.5, 'white'],  # Neutral point
    [1.0, 'green']   # For positive amounts
]

# Create the interactive plot
fig = px.scatter(
    daily_data,
    x='Day',
    y='Month',
    # size='Amount',
    color='Amount',
    color_continuous_scale=custom_color_scale,
    title='Interactive Day-wise Visualization of Amount',
    labels={'Day': 'Day of the Month', 'Month': 'Month'},
    hover_data={'Amount': ':.2f', 'Year': True}  # Format hover data
)

fig.update_layout(
    yaxis_title="Month",
    xaxis_title="Day of the Month",
    template="plotly_white",
    coloraxis_colorbar=dict(title="Amount"),
)

# Show the plot
fig.show()

# dashboard

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import calendar

# Load your Excel file
file_path = '/Users/akash_verma/Downloads/LEDGER REPORT29_12_2024.xlsx'
df = pd.read_excel(file_path, engine = 'openpyxl')
df.rename(columns = {"Doc Date":'Date'},inplace=True)
df.Date = pd.to_datetime(df.Date)
df = df[df.Narration.str.contains("Bill for the trade date")]

In [9]:
import pandas as pd
import numpy as np
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go



# Ensure the 'Date' column is in datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Extract year, month, and day
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

# Create the Dash app
app = Dash(__name__)

# Layout for the dashboard
app.layout = html.Div([
    html.H1("P&L Dashboard", style={'textAlign': 'center'}),
    html.Div([
        html.Label("Segment"),
        dcc.Dropdown(
            id='segment-dropdown',
            options=[{'label': 'Futures & Options', 'value': 'F&O'}],
            value='F&O'
        )
    ], style={'width': '20%', 'display': 'inline-block'}),
    html.Div([
        html.Label("P&L"),
        dcc.Dropdown(
            id='pnl-dropdown',
            options=[{'label': 'Combined', 'value': 'Combined'}],
            value='Combined'
        )
    ], style={'width': '20%', 'display': 'inline-block'}),
    html.Div([
        html.Label("Symbol"),
        dcc.Input(id='symbol-input', type='text', placeholder='eg: INFY')
    ], style={'width': '20%', 'display': 'inline-block'}),
    html.Div([
        html.Label("Date range"),
        dcc.DatePickerRange(
            id='date-picker',
            start_date="2024-04-01",
            end_date="2024-12-29"
        )
    ], style={'width': '30%', 'display': 'inline-block'}),
    html.Br(),
    html.Div([
        dcc.Graph(
            id='day-wise-heatmap',
            config={'displayModeBar': False}  # Hides toolbar for clean appearance
        )
    ])
])

# Callback to update the heatmap based on filters
@app.callback(
    Output('day-wise-heatmap', 'figure'),
    [
        Input('segment-dropdown', 'value'),
        Input('pnl-dropdown', 'value'),
        Input('symbol-input', 'value'),
        Input('date-picker', 'start_date'),
        Input('date-picker', 'end_date')
    ]
)
def update_heatmap(segment, pnl, symbol, start_date, end_date):
    # Filter data based on date range
    filtered_data = df[(df['Date'] >= pd.to_datetime(start_date)) & (df['Date'] <= pd.to_datetime(end_date))]
    
    # Pivot table for heatmap
    heatmap_data = filtered_data.pivot_table(
        index='Month',
        columns='Day',
        values='Amount',
        aggfunc='sum'
    )

    # Create the heatmap
    fig = go.Figure(
        data=go.Heatmap(
            z=heatmap_data,
            x=heatmap_data.columns,
            y=heatmap_data.index,
            colorscale=[[0.0, 'red'], [0.5, 'white'], [1.0, 'green']],  # Red for negative, Green for positive
            colorbar=dict(title="Amount"),
            xgap=1,  # Adds separation between days
            ygap=2   # Adds separation between months
        )
    )

    # Update layout for better separation and labels
    fig.update_layout(
        title='Day-wise P&L Heatmap',
        xaxis_title="Day",
        yaxis_title="Month",
        xaxis=dict(tickmode='linear', tick0=1, dtick=1),
        yaxis=dict(
            tickmode='array',
            tickvals=list(range(1, 13)),
            ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        ),
        template='plotly_white'
    )
    return fig

# Run the app
# Run on localhost
if __name__ == "__main__":
    app.run_server(debug=True,port = 1222, mode = 'external')

# Dash 2

In [42]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
import numpy as np
from datetime import date, timedelta

# -----------------------------------------------------------------------------
# Generate example data
# -----------------------------------------------------------------------------
def generate_example_data(start_date, end_date):
    """
    Returns a DataFrame of dates from start_date to end_date,
    with random P&L values (+/-).
    """
    dates = pd.date_range(start=start_date, end=end_date)
    # Random P&L between -5000 and 5000
    pnl_values = np.random.randint(-5000, 5000, len(dates))

    df = pd.DataFrame({"date": dates, "pnl": pnl_values})
    return df

# -----------------------------------------------------------------------------
# Prepare data for month-by-day layout
# -----------------------------------------------------------------------------
def make_month_day_df(df):
    """
    Adds columns for month_name and day_of_month. 
    """
    df = df.sort_values("date")
    df["month"] = df["date"].dt.month
    df["month_name"] = df["date"].dt.month_name()
    df["day_of_month"] = df["date"].dt.day

    # We'll store the P&L value in a separate column for color
    df["pnl_color"] = df["pnl"]
    return df

# -----------------------------------------------------------------------------
# Build the Dash app
# -----------------------------------------------------------------------------
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container(
    fluid=True,
    children=[
        html.H2("Futures & Options P&L Dashboard", className="my-3"),

        # Controls row
        dbc.Row([
            dbc.Col([
                html.Label("Segment"),
                dcc.Dropdown(
                    id="segment-dropdown",
                    options=[
                        {"label": "Futures & Options", "value": "FNO"},
                        {"label": "Equity", "value": "EQUITY"},
                        {"label": "Currency", "value": "CURRENCY"},
                    ],
                    value="FNO",
                    clearable=False,
                ),
            ], width=2),

            dbc.Col([
                html.Label("P&L"),
                dcc.Dropdown(
                    id="pnl-dropdown",
                    options=[
                        {"label": "Combined", "value": "combined"},
                        {"label": "Profit Only", "value": "profit"},
                        {"label": "Loss Only", "value": "loss"},
                    ],
                    value="combined",
                    clearable=False,
                ),
            ], width=2),

            dbc.Col([
                html.Label("Symbol"),
                dcc.Input(
                    id="symbol-input",
                    type="text",
                    placeholder="e.g. INFY",
                    value="",
                    style={"width": "100%"}
                ),
            ], width=2),

            dbc.Col([
                html.Label("Date Range"),
                dcc.DatePickerRange(
                    id="date-range",
                    min_date_allowed=date(2023, 1, 1),
                    max_date_allowed=date(2025, 12, 31),
                    start_date=date(2024, 4, 1),
                    end_date=date(2024, 12, 29),
                ),
            ], width=4),

            dbc.Col(
                dbc.Button("Go", id="go-button", color="primary", className="mt-4"),
                width=2
            ),
        ], className="mb-3"),

        # Graph row
        dbc.Row([
            dbc.Col([
                dcc.Graph(id="calendar-heatmap")
            ], width=12),
        ]),
    ]
)

# -----------------------------------------------------------------------------
# Callbacks
# -----------------------------------------------------------------------------
@app.callback(
    Output("calendar-heatmap", "figure"),
    [
        Input("segment-dropdown", "value"),
        Input("pnl-dropdown", "value"),
        Input("symbol-input", "value"),
        Input("date-range", "start_date"),
        Input("date-range", "end_date"),
        Input("go-button", "n_clicks"),
    ],
)
def update_calendar_heatmap(segment, pnl_filter, symbol, start_date, end_date, n_clicks):
    """
    Example flow:
      1. Generate/fetch daily P&L data for the chosen date range.
      2. Optionally filter out non-profit or non-loss records.
      3. Plot by month (y-axis) and day_of_month (x-axis).
    """
    if not start_date or not end_date:
        return {}

    # 1. Generate or fetch data
    import pandas as pd
    import matplotlib.pyplot as plt
    import calendar

    # Load your Excel file
    file_path = '/Users/akash_verma/Downloads/LEDGER REPORT29_12_2024 (2).xlsx'
    df = pd.read_excel(file_path, engine = 'openpyxl',)
    df.head()
    df.rename(columns = {"Doc Date":'Date'},inplace=True)
    df.Date = pd.to_datetime(df.Date,format= "%d/%m/%Y")
    df = df[df.Narration.str.contains("Bill for the trade date")]
    df['pnl'] = df['Amount']
    df['date'] = df['Date']

    # 2. (Optional) filter by P&L type
    if pnl_filter == "profit":
        df = df[df["pnl"] > 0]
    elif pnl_filter == "loss":
        df = df[df["pnl"] < 0]

    # 3. Transform for month-by-day layout
    df = make_month_day_df(df)

    # 4. Create Plotly figure
    # We'll order the months from January -> December.
    # Not all months may appear in the selected range, so
    # we’ll at least define the order for any that DO appear.
    unique_months = sorted(df["month"].unique())
    # Convert month number to month name for ordering
    month_order = [pd.to_datetime(str(m), format="%m").strftime("%B") 
                   for m in unique_months]

    fig = px.scatter(
        df,
        x="day_of_month",
        y="month_name",
        color="pnl_color",
        hover_data=["date", "pnl"],
        color_continuous_scale=["red", "white", "green"],
        title="Daily P&L by Month",
        height=600,
        category_orders={"month_name": month_order},
    )

    # Make the squares bigger
    fig.update_traces(marker=dict(size=18, symbol="square"))

    # Hide the colorbar if you prefer
    fig.update_layout(
        coloraxis_showscale=False,
        xaxis=dict(title="Day of Month", tickmode="linear"),
        yaxis=dict(title="Month"),
        plot_bgcolor="white",
    )

    return fig

# -----------------------------------------------------------------------------
# Run server
# -----------------------------------------------------------------------------
if __name__ == "__main__":
    app.run_server(debug=True,mode = 'external',port = 1222)

# dash 3

In [51]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
from datetime import date
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# ------------------------------------------------------------------
# Example Data
# ------------------------------------------------------------------
def generate_example_data(start_date, end_date):
    """
    Returns a DataFrame of dates from start_date to end_date,
    with random P&L values (+/-).
    """
    file_path = '/Users/akash_verma/Downloads/LEDGER REPORT29_12_2024 (2).xlsx'
    df = pd.read_excel(file_path, engine = 'openpyxl',)
    df.head()
    df.rename(columns = {"Doc Date":'Date'},inplace=True)
    df.Date = pd.to_datetime(df.Date,format= "%d/%m/%Y")
    df = df[df.Narration.str.contains("Bill for the trade date")]
    df['pnl'] = df['Amount']
    df['date'] = df['Date']

    return df

def make_calendar_df(df):
    """
    For each date, compute:
      - 'year' (int)
      - 'month' (int)
      - 'day_of_month' (1..31)
      - 'day_of_week' (Mon=0..Sun=6)
      - 'month_name' (January, February, etc.)
    """
    df = df.copy()
    df["year"] = df["date"].dt.year
    df["month"] = df["date"].dt.month
    df["month_name"] = df["date"].dt.month_name()
    df["day_of_month"] = df["date"].dt.day
    df["day_of_week"] = df["date"].dt.weekday  # Monday=0..Sunday=6
    return df

# ------------------------------------------------------------------
# Build Dash App
# ------------------------------------------------------------------
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container(
    fluid=True,
    children=[
        html.H2("F&O P&L Dashboard with Subplots", className="my-3"),

        dbc.Row([
            dbc.Col([
                html.Label("Segment"),
                dcc.Dropdown(
                    id="segment-dropdown",
                    options=[
                        {"label": "Futures & Options", "value": "FNO"},
                        {"label": "Equity", "value": "EQUITY"},
                        {"label": "Currency", "value": "CURRENCY"},
                    ],
                    value="FNO",
                    clearable=False,
                ),
            ], width=2),

            dbc.Col([
                html.Label("P&L"),
                dcc.Dropdown(
                    id="pnl-dropdown",
                    options=[
                        {"label": "Combined", "value": "combined"},
                        {"label": "Profit Only", "value": "profit"},
                        {"label": "Loss Only", "value": "loss"},
                    ],
                    value="combined",
                    clearable=False,
                ),
            ], width=2),

            dbc.Col([
                html.Label("Symbol"),
                dcc.Input(
                    id="symbol-input",
                    type="text",
                    placeholder="e.g. INFY",
                    value="",
                    style={"width": "100%"}
                ),
            ], width=2),

            dbc.Col([
                html.Label("Date Range"),
                dcc.DatePickerRange(
                    id="date-range",
                    min_date_allowed=date(2023, 1, 1),
                    max_date_allowed=date(2025, 12, 31),
                    start_date=date(2024, 4, 1),
                    end_date=date(2024, 12, 29),
                ),
            ], width=3),

            dbc.Col([
                html.Label("Month Filter"),
                dcc.Dropdown(
                    id="month-filter",
                    options=[],  # We’ll populate this in a callback
                    value=[],
                    multi=True,
                    placeholder="Select month(s)",
                ),
            ], width=3),
        ], className="mb-3"),

        dbc.Row([
            dbc.Col(
                dbc.Button("Go", id="go-button", color="primary", className="mt-2"),
                width=2
            ),
        ], className="mb-3"),

        dbc.Row([
            dbc.Col([
                dcc.Graph(id="subplots-graph", style={"height": "100%"})
            ], width=12),
        ]),
    ]
)

# ------------------------------------------------------------------
# 1) Populate the Month Filter options
# ------------------------------------------------------------------
@app.callback(
    Output("month-filter", "options"),
    [
        Input("date-range", "start_date"),
        Input("date-range", "end_date"),
    ],
)
def update_month_filter_options(start_date, end_date):
    """
    Based on the date range, figure out which months (1..12) actually appear
    in that range, and produce e.g. [{'label': 'April', 'value': 4}, ...].
    """
    if not start_date or not end_date:
        return []

    # Generate a small DF just to see which months are in the range
    df_temp = pd.date_range(start=start_date, end=end_date, freq="D")
    months_in_range = sorted(list(df_temp.month.unique()))

    options = []
    for m in months_in_range:
        month_name = pd.to_datetime(str(m), format="%m").strftime("%B")
        options.append({"label": month_name, "value": m})

    return options

# ------------------------------------------------------------------
# 2) Build Subplots Graph
# ------------------------------------------------------------------
@app.callback(
    Output("subplots-graph", "figure"),
    [
        Input("segment-dropdown", "value"),
        Input("pnl-dropdown", "value"),
        Input("symbol-input", "value"),
        Input("date-range", "start_date"),
        Input("date-range", "end_date"),
        Input("month-filter", "value"),
        Input("go-button", "n_clicks"),
    ],
)
def update_subplots(segment, pnl_filter, symbol, start_date, end_date,
                    months_selected, n_clicks):
    """
    - Generate random P&L data for the date range
    - Filter profit/loss if needed
    - Use the chosen month filter (if user has selected some months)
    - Create a subplots figure, each subplot = one month
      (day_of_month on x-axis, day_of_week on y-axis, colored by P&L).
    """
    # Safety check
    if not start_date or not end_date:
        return go.Figure()

    # 1) Get your data (dummy random data here)
    df = generate_example_data(start_date, end_date)

    # 2) Filter by P&L
    if pnl_filter == "profit":
        df = df[df["pnl"] > 0]
    elif pnl_filter == "loss":
        df = df[df["pnl"] < 0]

    # 3) Prepare for calendar style
    df = make_calendar_df(df)

    # 4) If user selected certain months, filter them
    #    If user hasn't chosen anything in the Month Filter, show all months
    if months_selected and len(months_selected) > 0:
        df = df[df["month"].isin(months_selected)]

    # 5) Figure out which months we actually need to plot (unique months in the DF)
    unique_months = sorted(df["month"].unique())
    if len(unique_months) == 0:
        # If there's no data, return an empty figure
        return go.Figure()

    # # 6) Create subplots. We'll do at most 4 rows x 3 columns = 12 months.
    # rows = 4
    # cols = 3
    # # Increase spacing to prevent overlap
    # fig = make_subplots(
    #     rows=rows, 
    #     cols=cols,
    #     shared_xaxes=False, 
    #     shared_yaxes=False,
    #     subplot_titles=[
    #         pd.to_datetime(str(m), format="%m").strftime("%B")
    #         for m in unique_months
    #     ],
    #     horizontal_spacing=0.08,  # Adjust as needed (0.02 ~ 0.1)
    #     vertical_spacing=0.12,    # Adjust as needed (0.02 ~ 0.2)
    # )

    # # 7) For each month, add a scatter trace to the figure
    # index = 0
    # for m in unique_months:
    #     row = (index // cols) + 1
    #     col = (index % cols) + 1

    #     df_m = df[df["month"] == m].copy()
    #     trace = go.Scatter(
    #         x=df_m["day_of_month"],
    #         y=df_m["day_of_week"],
    #         mode="markers",
    #         marker=dict(
    #             size=15,
    #             symbol="square",
    #             color=df_m["pnl"],
    #             colorscale=["red","white","green"],
    #             cmin=-5000,  # or use df["pnl"].min()
    #             cmax=5000,   # or use df["pnl"].max()
    #             showscale=False
    #         ),
    #         text=[
    #             f"{d.strftime('%Y-%m-%d')}<br>P&L: {p}" 
    #             for d, p in zip(df_m["date"], df_m["pnl"])
    #         ],
    #         hoverinfo="text",
    #         name=f"Month {m}",
    #     )
    #     fig.add_trace(trace, row=row, col=col)
    #     index += 1

    # # 8) Adjust layout for each subplot
    # #    For day_of_week, Monday=0..Sunday=6. Let's label them:
    # day_of_week_labels = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]

    # for i in range(len(unique_months)):
    #     row = (i // cols) + 1
    #     col = (i % cols) + 1
    #     axis_id_x = f"xaxis{(i+1) if (i+1)>1 else ''}"
    #     axis_id_y = f"yaxis{(i+1) if (i+1)>1 else ''}"

    #     fig["layout"][axis_id_x].update(
    #         title_text="Day of Month",
    #         range=[0.5, 31.5],
    #         dtick=5,       # Show tick every 5 days
    #         showgrid=False,
    #     )
    #     fig["layout"][axis_id_y].update(
    #         title_text="Day of Week",
    #         tickmode="array",
    #         tickvals=list(range(7)),
    #         ticktext=day_of_week_labels,
    #         showgrid=False,
    #     )

    # # Hide unused subplots if there are fewer than 12
    # total_subplots = rows * cols
    # for i in range(len(unique_months), total_subplots):
    #     axis_id_x = f"xaxis{(i+1) if (i+1)>1 else ''}"
    #     axis_id_y = f"yaxis{(i+1) if (i+1)>1 else ''}"
    #     fig["layout"][axis_id_x].update(visible=False)
    #     fig["layout"][axis_id_y].update(visible=False)

    # # 9) Final layout tweaks
    # #    Increase the figure size and add margins
    # fig.update_layout(
    #     title=f"Subplots by Month – Segment: {segment}, Symbol: {symbol}",
    #     showlegend=False,
    #     width=1300,   # Increase width
    #     height=1200,  # Increase height
    #     margin=dict(l=60, r=40, t=80, b=60),
    #     font=dict(size=12),  # Adjust overall font size
    # )
    import plotly.graph_objs as go
    from plotly.subplots import make_subplots
    import pandas as pd

    # -- Create or load your df with columns: [date, month, day_of_month, day_of_week, pnl] --
    # unique_months = sorted(df["month"].unique())

    rows = 6
    cols = 2
    fig = make_subplots(
        rows=rows, 
        cols=cols,
        shared_xaxes=False, 
        shared_yaxes=False,
        subplot_titles=[
            pd.to_datetime(str(m), format="%m").strftime("%B")
            for m in unique_months
        ],
        horizontal_spacing=0.08,
        vertical_spacing=0.12,
    )

    index = 0
    for m in unique_months:
        row = (index // cols) + 1
        col = (index % cols) + 1

        df_m = df[df["month"] == m].copy()
        hover_text = [
            f"{d.strftime('%Y-%m-%d')}<br>P&L: {p}" 
            for d, p in zip(df_m["date"], df_m["pnl"])
        ]

        # Heatmap trace
        trace = go.Heatmap(
            x=df_m["day_of_month"],
            y=df_m["day_of_week"],
            z=df_m["pnl"],
            text=hover_text,
            hoverinfo="text",
            colorscale=["red", "white", "green"],
            zmin=-5000,
            zmax=5000,
            hoverongaps=False,
            showscale=False,  # or True if you want a colorbar per subplot
            name=f"Month {m}",
        )
        fig.add_trace(trace, row=row, col=col)

        index += 1

    day_of_week_labels = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
    total_plots = len(unique_months)

    for i in range(total_plots):
        # 0-based index i => subplot # is i+1
        # row, col for subplot
        row = (i // cols) + 1
        col = (i % cols) + 1
        
        # Axis names in layout
        # For subplot 1 => xaxis, yaxis
        # For subplot 2 => xaxis2, yaxis2
        # etc.
        xaxis_id = f"xaxis{(i+1) if (i+1) > 1 else ''}"
        yaxis_id = f"yaxis{(i+1) if (i+1) > 1 else ''}"

        # Set the day_of_month axis range
        fig.layout[xaxis_id].update(
            title_text="Day of Month",
            range=[0.5, 31.5],  # so that day 1..31 are centered
            dtick=5,
            showgrid=False
        )
        # Set the day_of_week axis range
        fig.layout[yaxis_id].update(
            title_text="Day of Week",
            tickmode="array",
            tickvals=list(range(7)),
            ticktext=day_of_week_labels,
            range=[-0.5, 6.5],  # so that 0..6 are centered
            showgrid=False
        )

        # --- Force square cells here ---
        # 'scaleanchor' must match the X-axis name in the layout: "xaxis1" => "scaleanchor='x1'"
        # For the 1st subplot, "xaxis" is 'xaxis' but we can pass 'x1' to scaleanchor.
        # A simple trick: scaleanchor='x' for the first subplot, scaleanchor='x2' for the 2nd, etc.
        # We'll map subplot i+1 => anchor 'x{i+1}' if i+1>1, else 'x'
        anchor_name = f"x{i+1}" if (i+1) > 1 else "x"
        fig.layout[yaxis_id].update(
            scaleanchor=anchor_name,  # lock y-scale to x
            scaleratio=1              # 1:1 aspect ratio
        )

    # Hide the unused subplots if less than rows*cols
    for i in range(len(unique_months), rows*cols):
        xaxis_id = f"xaxis{(i+1) if (i+1) > 1 else ''}"
        yaxis_id = f"yaxis{(i+1) if (i+1) > 1 else ''}"
        fig.layout[xaxis_id].update(visible=False)
        fig.layout[yaxis_id].update(visible=False)

    fig.update_layout(
        title="Heatmap by Month (Squares)",
        showlegend=False,
        width=1300,
        height=1200,
        margin=dict(l=60, r=40, t=80, b=60),
        font=dict(size=12),
    )

 

    return fig


# ------------------------------------------------------------------
# Run
# ------------------------------------------------------------------
if __name__ == "__main__":
    app.run_server(debug=True)

---------------------------------------------------------------------------
UnboundLocalError                         Traceback (most recent call last)
Input In [51], in update_subplots(
    segment='FNO',
    pnl_filter='combined',
    symbol='',
    start_date='2024-04-01',
    end_date='2024-12-29',
    months_selected=[4],
    n_clicks=None
)
    209 unique_months = sorted(df["month"].unique())
    210 if len(unique_months) == 0:
    211     # If there's no data, return an empty figure
--> 212     return go.Figure()
        go = <module 'plotly.graph_objs' from '/Users/akash_verma/miniconda3/envs/epat/lib/python3.8/site-packages/plotly/graph_objs/__init__.py'>
    214 # # 6) Create subplots. We'll do at most 4 rows x 3 columns = 12 months.
    215 # rows = 4
    216 # cols = 3
   (...)
    301 #     font=dict(size=12),  # Adjust overall font size
    302 # )
    303 import plotly.graph_objs as go

UnboundLocalError: local variable 'go' referenced before assignment

----------------